## Install

In [1]:
%%capture
!pip install "transformers==4.44.2" "accelerate==0.34.2" "bitsandbytes" "datasets" \
             "rouge-score==0.1.2" "nltk==3.9.1" "tqdm==4.66.5" "safetensors>=0.4.3" \
             git+https://github.com/AIPHES/emnlp19-moverscore.git


In [2]:
from google.colab import drive
import os
drive.mount("/content/drive")
path = "/content/drive/MyDrive/11785-project-medred"

os.chdir(path)


Mounted at /content/drive


In [3]:
from huggingface_hub import login
login()


In [4]:
!pip install "numpy<2.0.0" --force-reinstall
!pip install --no-binary=:all: pyemd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 125.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 r

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... canceled
  ERROR: Operation cancelled by user
^C


In [1]:
!pip install pyemd
!pip install textstat

  Using cached pyemd-1.0.0.tar.gz (87 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyemd: filename=pyemd-1.0.0-cp312-cp312-linux_x86_64.whl size=742847 sha256=a961a3597b8b7fd1288ff055ad9f971dad807be9bc8dc097dd4d2026e7fd1a10
  Stored in directory: /root/.cache/pip/wheels/bf/7d/e0/84ae1a3c2e45898a01b400c288b56a601c03fd36f2a4d060bf
Successfully built pyemd
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.1 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/AIPHES/emnlp19-moverscore.git

  Cloning https://github.com/AIPHES/emnlp19-moverscore.git to /tmp/pip-req-build-wxfgodfu
  Running command git clone --filter=blob:none --quiet https://github.com/AIPHES/emnlp19-moverscore.git /tmp/pip-req-build-wxfgodfu
  Resolved https://github.com/AIPHES/emnlp19-moverscore.git to commit 0459a3b3b3bd73baa0cc515a355228ee5a2887e0
  Preparing metadata (setup.py) ... done


In [3]:
import os

path = "/content/drive/MyDrive/11785-project-medred"

os.chdir(path)

## Import

In [4]:
from moverscore_v2 import get_idf_dict, word_mover_score

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [5]:
import os, gc, json, random, time, re, string, sys
from pathlib import Path
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional, Dict, Any

import numpy as np
import pandas as pd
from tqdm import tqdm
import textstat
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)

In [6]:
# Optional readability and MoverScore
try:
    import textstat
except Exception:
    textstat = None

try:
    from moverscore_v2 import word_mover_score, get_idf_dict
    HAVE_MOVERSCORE = True
except Exception as e:
    print("WARNING: could not import moverscore_v2:", e)
    HAVE_MOVERSCORE = False

## Config

In [7]:
# ------------------------------------------------------------------
# Small tokenizer compatibility patch for MoverScore (DistilBERT)
# ------------------------------------------------------------------
def _patch_tokenizer_max_len():
    try:
        import transformers
        for name in ("PreTrainedTokenizer", "PreTrainedTokenizerFast"):
            cls = getattr(transformers, name, None)
            if cls is not None and not hasattr(cls, "max_len"):
                cls.max_len = property(lambda self: getattr(self, "model_max_length", 512))
    except Exception as e:
        print("Tokenizer patch for max_len failed:", type(e).__name__, e)

_patch_tokenizer_max_len()

# ============================================================
#  CONFIG
# ============================================================
PROJECT_ROOT = "/content/drive/MyDrive/11785-project-medred"
MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"

class CFG:
    LOCAL_MODEL_DIR = os.getenv("LLAMA_MODEL_DIR", MODEL_ID)

    MEDRED_CSV = os.getenv(
        "MEDRED_CSV",
        f"{PROJECT_ROOT}/medredqa_test.csv",
    )

    OUT_DIR = Path(os.getenv("OUT_DIR", f"{PROJECT_ROOT}/outputs"))
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    MAX_NEW_TOKENS = int(os.getenv("MAX_NEW_TOKENS", 160))
    DO_SAMPLE = False

    MAX_INPUT_TOKENS    = int(os.getenv("MAX_INPUT_TOKENS", 1024))
    BATCH_TOKENS_BUDGET = int(os.getenv("BATCH_TOKENS_BUDGET", 10**9))
    MAX_BATCH_SAMPLES   = int(os.getenv("MAX_BATCH_SAMPLES", 1))
    SAFETY              = float(os.getenv("BATCH_SAFETY", 0.9))

    SYSTEM_PROMPT = "You are a careful medical assistant. Answer clearly and concisely for lay readers."

    # Fixed eval design
    N_EVAL_TOTAL = int(os.getenv("N_EVAL_TOTAL", "1000"))   # total MedRed eval ids
    SHARD_TOTAL  = int(os.getenv("SHARD_TOTAL", "4"))       # 4 shards of ≈250 each
    SHARD_INDEX  = int(os.getenv("SHARD_INDEX", "1"))       # 0,1,2,3 for 4×250

# Reflexion hyperparams
MAX_STEPS = int(os.getenv("MAX_STEPS", 6))
MAX_REFLECT_ATTEMPTS = int(os.getenv("MAX_REFLECT_ATTEMPTS", 3))

# ============================================================
#  SEEDING & ENV
# ============================================================
SEED = 42
os.environ.setdefault("PYTHONHASHSEED", str(SEED))
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True
try:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
except Exception:
    pass

os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["JAX_PLATFORMS"] = "cpu"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

# ============================================================
#  LOAD MODEL (4-bit Llama-3.1-8B Instruct)
# ============================================================
bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tok = AutoTokenizer.from_pretrained(
    CFG.LOCAL_MODEL_DIR,
    local_files_only=False,
    use_fast=True,
)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "left"

model = AutoModelForCausalLM.from_pretrained(
    CFG.LOCAL_MODEL_DIR,
    device_map={"": "cuda:0"},
    low_cpu_mem_usage=True,
    quantization_config=bnb,
    torch_dtype=torch.float16,
    attn_implementation="eager",
)
model.eval()
model.config.pad_token_id = tok.pad_token_id

DEVICE = model.device
print("Model on", DEVICE)

# ============================================================
#  MEDRED DATAFRAME
# ============================================================
df_raw = pd.read_csv(CFG.MEDRED_CSV)
df = pd.DataFrame({
    "id": df_raw.iloc[:, 0].astype(str),
    "title": df_raw["Title"].fillna("").astype(str),
    "body": df_raw["Body"].fillna("").astype(str),
    "reference": df_raw["Response"].fillna("").astype(str).str.strip(),
})
df["query"] = (df["title"] + "\n\n" + df["body"]).str.strip()
df = df[df["query"].str.len() > 0].reset_index(drop=True)
df["id"] = df["id"].astype(str)

print("Full MedRed size:", len(df))

# ============================================================
#  FIXED 1000-ID SAMPLING + 4×250 SHARDS
# ============================================================
IDS_1000_PATH = Path(PROJECT_ROOT) / "medred_ids_1000_fixed.csv"

if IDS_1000_PATH.exists():
    ids_1000_df = pd.read_csv(IDS_1000_PATH, dtype={"id": str})
    ids_1000 = ids_1000_df["id"].astype(str).tolist()
    print(f"Loaded fixed eval ids from {IDS_1000_PATH} (n={len(ids_1000)})")
else:
    # Sample once with fixed seed and save to disk
    n_total = min(CFG.N_EVAL_TOTAL, len(df))
    ids_1000 = (
        df[["id"]]
        .drop_duplicates()
        .sample(n=n_total, random_state=SEED, replace=False)
        ["id"].astype(str).tolist()
    )
    pd.DataFrame({"id": ids_1000}).to_csv(IDS_1000_PATH, index=False)
    print(f"Sampled and saved fixed eval ids to {IDS_1000_PATH} (n={len(ids_1000)})")

if CFG.SHARD_TOTAL <= 0:
    raise ValueError("SHARD_TOTAL must be >= 1")

if CFG.SHARD_INDEX < 0 or CFG.SHARD_INDEX >= CFG.SHARD_TOTAL:
    raise ValueError(f"SHARD_INDEX must be in [0, {CFG.SHARD_TOTAL-1}]")

shard_size = len(ids_1000) // CFG.SHARD_TOTAL
start = CFG.SHARD_INDEX * shard_size
end   = (CFG.SHARD_INDEX + 1) * shard_size if CFG.SHARD_INDEX < CFG.SHARD_TOTAL - 1 else len(ids_1000)
shard_ids = ids_1000[start:end]
print(f"Using shard_index={CFG.SHARD_INDEX} of {CFG.SHARD_TOTAL} → rows [{start}:{end}) → {len(shard_ids)} ids")

# Subset df to this shard, preserving shard_ids order
order_map = {id_: i for i, id_ in enumerate(shard_ids)}
df_reflex_in = df[df["id"].isin(shard_ids)].copy()
df_reflex_in["__order"] = df_reflex_in["id"].map(order_map)
df_reflex_in = df_reflex_in.sort_values("__order").drop(columns=["__order"]).reset_index(drop=True)

print("Reflexion subset size (this shard):", len(df_reflex_in))


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Model on cuda:0
Full MedRed size: 5099
Loaded fixed eval ids from /content/drive/MyDrive/11785-project-medred/medred_ids_1000_fixed.csv (n=1000)
Using shard_index=1 of 4 → rows [250:500) → 250 ids
Reflexion subset size (this shard): 250


## Baseline

In [ ]:
# ============================================================
#  BASELINE ONLY on MedRed 1000
#  - Fix 1000 ids, take shard of 500
#  - Batched generation, EM on full answer
#  - Adds self-reported confidence + Brier + ECE
# ============================================================

import os, gc, json, re, string, random
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional

from pathlib import Path

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from rouge_score import rouge_scorer

# Optional readability
try:
    import textstat
except Exception:
    textstat = None

# ============================================================
# 1) Global settings: sampling, ECE, paths
# ============================================================

SEED = int(os.getenv("MEDRED_SEED", "42"))

ROUGE_TAU_FOR_ECE = float(os.getenv("ECE_ROUGE_TAU", "0.2"))
ECE_N_BINS = int(os.getenv("ECE_N_BINS", "10"))

ID_PATH_1000 = Path("/content/drive/MyDrive/11785-project-medred/medred_ids_1000_fixed.csv")

EVAL_ROOT = Path(CFG.OUT_DIR) / "medred_eval_1000"
EVAL_ROOT.mkdir(parents=True, exist_ok=True)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ============================================================
# 2) Helper functions
# ============================================================

def normalize_ws(s: str) -> str:
    return " ".join((s or "").split())

def extract_confidence(text: str) -> float:
    """
    Parse self-reported confidence from the model output.
    Looks for 'Confidence: p' (with p in [0,1] or [0,100]).
    """
    if not text:
        return np.nan
    m = re.search(r"confidence\s*[:=]\s*([0-9]+\.?[0-9]*|\.[0-9]+)",
                  text,
                  flags=re.IGNORECASE)
    if not m:
        return np.nan
    try:
        c = float(m.group(1))
        # If looks like percentage, convert to [0,1]
        if c > 1.0 and c <= 100.0:
            c = c / 100.0
        c = max(0.0, min(1.0, c))
        return float(c)
    except Exception:
        return np.nan

def strip_confidence_lines(text: str) -> str:
    """
    Remove 'Confidence: ...' lines so ROUGE/readability
    are computed only on the actual answer text.
    """
    if not text:
        return ""
    lines = text.splitlines()
    kept = [ln for ln in lines
            if not re.match(r"^\s*confidence\s*[:=]", ln, flags=re.IGNORECASE)]
    return "\n".join(kept).strip()

def compute_brier(correct: np.ndarray, conf: np.ndarray) -> float:
    """
    Brier score for binary outcomes.
    correct: array-like of 0/1
    conf   : array-like of probabilities in [0,1], may contain NaN
    """
    conf = np.asarray(conf, dtype=float)
    correct = np.asarray(correct, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    return float(np.mean((p - y) ** 2))

def compute_ece(correct: np.ndarray, conf: np.ndarray, n_bins: int = 10) -> float:
    """
    Expected Calibration Error using scalar confidence.
    correct: 0/1
    conf   : in [0,1], may contain NaN
    """
    conf = np.asarray(conf, dtype=float)
    correct = np.asarray(correct, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    N = len(p)
    bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for b in range(n_bins):
        left = bin_edges[b]
        right = bin_edges[b + 1]
        if b < n_bins - 1:
            in_bin = (p >= left) & (p < right)
        else:
            in_bin = (p >= left) & (p <= right + 1e-8)
        if not np.any(in_bin):
            continue
        acc_b = float(y[in_bin].mean())
        conf_b = float(p[in_bin].mean())
        ece += (in_bin.sum() / N) * abs(acc_b - conf_b)
    return float(ece)

# ROUGE scorer (for multiprocessing)
def _init_scorer():
    global _SCORER
    _SCORER = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)

def _r1_pair(args):
    ref, hyp = args
    return _SCORER.score(ref, hyp)["rouge1"].fmeasure

def normalize_simple(s: Optional[str]) -> str:
    """
    EM-style normalization (same as earlier MedRed code).
    """
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def ws(text):
        return " ".join(text.split())
    txt = (s or "").lower()
    txt = remove_articles(txt)
    txt = remove_punc(txt)
    return ws(txt)

def EM(ans: str, key: str) -> bool:
    return normalize_simple(ans) == normalize_simple(key)

# ============================================================
# 3) Fix the 1000 MedRed ids once, then pick 500-shard
# ============================================================

df = df.copy()
df["id"] = df["id"].astype(str)

if ID_PATH_1000.exists():
    ids_1000 = pd.read_csv(ID_PATH_1000, dtype={"id": str})["id"].astype(str).tolist()
    print(f"[SAMPLING] Loaded fixed 1000 ids from {ID_PATH_1000}")
else:
    print(f"[SAMPLING] Creating new 1000-id subset and saving to {ID_PATH_1000}")
    df_sample = df.sample(n=1000, random_state=SEED).reset_index(drop=True)
    ids_1000 = df_sample["id"].astype(str).tolist()
    pd.DataFrame({"id": ids_1000}).to_csv(ID_PATH_1000, index=False)

assert len(ids_1000) == 1000, f"Expected 1000 ids, got {len(ids_1000)}"

SHARD_SIZE = 500
SHARD_INDEX = int(os.getenv("SHARD_INDEX", "1"))  # 0: first 500, 1: second 500

start = SHARD_INDEX * SHARD_SIZE
end = min(start + SHARD_SIZE, len(ids_1000))
ids_shard = ids_1000[start:end]
print(f"[SHARD] Using shard_index={SHARD_INDEX} with rows [{start}:{end}) → {len(ids_shard)} ids")

order_map = {id_: i for i, id_ in enumerate(ids_shard)}

df_shard = (
    df[df["id"].isin(ids_shard)]
    .assign(__order=lambda d: d["id"].map(order_map))
    .sort_values("__order")
    .drop(columns=["__order"])
    .reset_index(drop=True)
)

print("[SHARD] df_shard size:", len(df_shard))

# ============================================================
# 4) Batching/generation helpers
# ============================================================

def make_batches_by_padded_cost(prompt_lens, budget_tokens, max_new, max_batch_samples, safety=0.9):
    batches, cur = [], []
    cur_max = 0
    eff = int(budget_tokens * safety)
    for i, L in enumerate(prompt_lens):
        new_max = max(cur_max, L)
        bs_next = len(cur) + 1
        cost = bs_next * (new_max + max_new)
        if cur and (cost > eff or len(cur) >= max_batch_samples):
            batches.append(cur)
            cur = [i]
            cur_max = L
        else:
            cur.append(i)
            cur_max = new_max
    if cur:
        batches.append(cur)
    return batches

def left_pad_collate(iid_list, device):
    maxL = max(len(x) for x in iid_list)
    pad_id = tok.pad_token_id
    input_ids = torch.full((len(iid_list), maxL), pad_id, dtype=torch.long)
    attn_mask = torch.zeros((len(iid_list), maxL), dtype=torch.bool)
    for r, ids_r in enumerate(iid_list):
        L = len(ids_r)
        input_ids[r, -L:] = torch.as_tensor(ids_r, dtype=torch.long)
        attn_mask[r,  -L:] = True
    return input_ids.to(device, non_blocking=True), attn_mask.to(device, non_blocking=True)

# ============================================================
# 5) Baseline generator: batched inference + metrics + calibration
# ============================================================

def generate_for_model(model_label: str,
                       df_in: pd.DataFrame,
                       prompts: List[str]) -> (pd.DataFrame, Dict[str, Any]):
    """
    Shared generator: runs batched inference, then
    computes ROUGE-1, readability, Brier, ECE.
    """
    assert len(prompts) == len(df_in)

    enc = tok(
        prompts,
        add_special_tokens=True,
        padding=False,
        truncation=True,
        max_length=CFG.MAX_INPUT_TOKENS,
        return_tensors=None,
    )
    input_ids_list = enc["input_ids"]
    lens = [len(x) for x in input_ids_list]

    order = np.argsort(lens).tolist()
    row_sorted = [i for i in order]
    iids_sorted = [input_ids_list[i] for i in order]
    lens_sorted = [lens[i] for i in order]

    batches = make_batches_by_padded_cost(
        lens_sorted,
        CFG.BATCH_TOKENS_BUDGET,
        CFG.MAX_NEW_TOKENS,
        CFG.MAX_BATCH_SAMPLES,
        safety=CFG.SAFETY,
    )

    # IMPORTANT: don't add extra stop tokens like "\n\n" that could
    # truncate before the "Confidence:" line.
    eos_token_ids = [tok.eos_token_id]
    try:
        eot_id = tok.convert_tokens_to_ids("<|eot_id|>")
        if isinstance(eot_id, int) and eot_id != tok.eos_token_id:
            eos_token_ids.append(eot_id)
    except Exception:
        pass

    pred_rows = []
    with torch.inference_mode():
        for b_ix, idxs in enumerate(tqdm(batches, desc=f"[{model_label}] generating")):
            batch_iids = [iids_sorted[i] for i in idxs]
            row_ids = [row_sorted[i] for i in idxs]

            input_ids, attn_mask = left_pad_collate(batch_iids, model.device)

            out = model.generate(
                input_ids=input_ids,
                attention_mask=attn_mask,
                max_new_tokens=CFG.MAX_NEW_TOKENS,
                do_sample=False,
                eos_token_id=eos_token_ids,
                pad_token_id=tok.pad_token_id,
                use_cache=True,
            )

            new_tokens = out[:, input_ids.shape[1]:]
            texts = tok.batch_decode(new_tokens, skip_special_tokens=True)

            for row_id, txt in zip(row_ids, texts):
                full_text = txt.strip()
                conf = extract_confidence(full_text)
                pred_clean = strip_confidence_lines(full_text)
                pred_rows.append({
                    "row_idx": row_id,
                    "prediction_raw": full_text,
                    "prediction": pred_clean,
                    "confidence": conf,
                })

            del input_ids, attn_mask, out, new_tokens, texts
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            gc.collect()

    preds_df = pd.DataFrame(pred_rows).sort_values("row_idx").reset_index(drop=True)
    assert len(preds_df) == len(df_in)

    eval_df = df_in.reset_index(drop=True).copy()
    eval_df = pd.concat(
        [eval_df, preds_df.drop(columns=["row_idx"])],
        axis=1
    )

    eval_df["reference_clean"]   = eval_df["reference"].map(normalize_ws)
    eval_df["prediction_clean"]  = eval_df["prediction"].fillna("").map(normalize_ws)
    # Exact-match EM on normalized strings
    eval_df["em"] = [
        EM(pred, ref)
        for pred, ref in zip(eval_df["prediction_clean"], eval_df["reference_clean"])
    ]

    print(f"[{model_label}] eval rows:", len(eval_df))

    # ROUGE-1(F1)
    pairs = list(zip(eval_df["reference_clean"], eval_df["prediction_clean"]))
    with Pool(processes=max(1, cpu_count() - 1), initializer=_init_scorer) as pool:
        r1 = np.array(
            list(
                tqdm(
                    pool.imap(_r1_pair, pairs),
                    total=len(eval_df),
                    desc=f"[{model_label}] ROUGE-1(F1)",
                )
            ),
            dtype=np.float32,
        )
    eval_df["rouge1_f1"] = r1

    # Readability
    if textstat is not None:
        try:
            eval_df["readability_fre"]   = eval_df["prediction_clean"].map(textstat.flesch_reading_ease)
            eval_df["readability_fkgl"]  = eval_df["prediction_clean"].map(textstat.flesch_kincaid_grade)
            eval_df["readability_smog"]  = eval_df["prediction_clean"].map(textstat.smog_index)
        except Exception as e:
            print(f"WARNING: [{model_label}] readability computation failed:", e)
            eval_df["readability_fre"]   = np.nan
            eval_df["readability_fkgl"]  = np.nan
            eval_df["readability_smog"]  = np.nan
    else:
        eval_df["readability_fre"]   = np.nan
        eval_df["readability_fkgl"]  = np.nan
        eval_df["readability_smog"]  = np.nan

    # ---- Calibration targets ----
    r1_vals   = eval_df["rouge1_f1"].values.astype(float)
    em_vals   = eval_df["em"].astype(float).values      # 0/1 from EM
    conf_vals = eval_df["confidence"].values.astype(float)

    # 1) ROUGE-based proxy: 1 if ROUGE-1 ≥ tau, else 0 (NaNs → 0)
    correct_proxy = np.where(
        np.isnan(r1_vals),
        0.0,
        (r1_vals >= ROUGE_TAU_FOR_ECE).astype(float),
    )

    brier_rouge = compute_brier(correct_proxy, conf_vals)
    ece_rouge   = compute_ece(correct_proxy, conf_vals, n_bins=ECE_N_BINS)

    # 2) EM-based calibration
    brier_em = compute_brier(em_vals, conf_vals)
    ece_em   = compute_ece(em_vals, conf_vals, n_bins=ECE_N_BINS)

    summary = {
        "model": model_label,
        "n": int(len(eval_df)),

        # Core accuracy-style metrics
        "rouge1_f1_mean": float(np.nanmean(r1_vals)),
        "rouge1_f1_std":  float(np.nanstd(r1_vals)),
        "em_mean":        float(np.nanmean(em_vals)),

        # Readability
        "readability_fre_mean":   float(np.nanmean(eval_df["readability_fre"])),
        "readability_fkgl_mean":  float(np.nanmean(eval_df["readability_fkgl"])),
        "readability_smog_mean":  float(np.nanmean(eval_df["readability_smog"])),

        # Calibration (ROUGE-based proxy)
        "brier_rouge_proxy": float(brier_rouge),
        "ece_rouge_proxy":   float(ece_rouge),
        "ece_rouge_tau":     float(ROUGE_TAU_FOR_ECE),
        "ece_n_bins":        int(ECE_N_BINS),

        # Calibration (EM-based)
        "brier_em": float(brier_em),
        "ece_em":   float(ece_em),
    }

    return eval_df, summary

# ============================================================
# 6) Baseline prompt (enforces Answer + Confidence lines)
# ============================================================

def build_baseline_prompt(q: str) -> str:
    """
    Baseline prompt: two-line structured output.

    We enforce:
      Answer: <...>
      Confidence: p
    where p is in [0,1].
    """
    user_content = f"""You are answering a clinical question based on a note.

Clinical note + question:
{q}

Please respond in EXACTLY this format, on two lines:

Answer: <your best answer here>
Confidence: p

where p is a single number between 0 and 1 (for example: 0.72).
Do not add any extra text before or after these two lines.
"""

    msgs = [
        {"role": "system", "content": CFG.SYSTEM_PROMPT},
        {"role": "user",   "content": user_content},
    ]
    try:
        return tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    except Exception:
        return (
            f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{CFG.SYSTEM_PROMPT}\n"
            f"<|eot_id|><|start_header_id|>user<|end_header_id|>\n{user_content}\n"
            f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        )

print("Baseline setup complete. Ready to run baseline on this shard.")

# ============================================================
# 7) BASELINE ONLY on this shard
# ============================================================

prompts_baseline = [build_baseline_prompt(q) for q in df_shard["query"].tolist()]

eval_base, summary_base = generate_for_model(
    model_label=f"baseline_shard{SHARD_INDEX}",
    df_in=df_shard,
    prompts=prompts_baseline,
)

base_dir = EVAL_ROOT / f"baseline_shard{SHARD_INDEX}"
base_dir.mkdir(parents=True, exist_ok=True)
eval_base.to_csv(base_dir / "per_example_scores.csv", index=False)
with open(base_dir / "summary.json", "w") as f:
    json.dump(summary_base, f, indent=2)

print("\n[BASELINE] Summary:")
print(json.dumps(summary_base, indent=2))


## Baseline + CoT

In [ ]:
# ============================================================
#  MedRed 1000-sample: Baseline + Baseline+CoT (original logic)
#  - Fix 1000 ids, take shard of 500
#  - Baseline: batched generation, EM on full answer
#  - CoT: agent-style, short EM answer + long rationaleK
#  - Adds calibration metrics (Brier/ECE)
# ============================================================

import os, gc, json, re, string, random
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional

from pathlib import Path

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from rouge_score import rouge_scorer

# Optional readability
try:
    import textstat
except Exception:
    textstat = None

# ============================================================
# 0) Assumptions
# ============================================================
# Assumes you already have:
#   - CFG, tok, model defined (config + load model cell)
#   - df with columns: id (str/int), query (str), reference (str)
# We just make a safe copy here:
df = df.copy()
df["id"] = df["id"].astype(str)

# ============================================================
# 1) Global settings: sampling, ECE, paths
# ============================================================

SEED = int(os.getenv("MEDRED_SEED", "42"))

# For ROUGE-based calibration (optional)
ROUGE_TAU_FOR_ECE = float(os.getenv("ECE_ROUGE_TAU", "0.2"))
ECE_N_BINS = int(os.getenv("ECE_N_BINS", "10"))

# Where to store the fixed 1000 MedRed ids
ID_PATH_1000 = Path("/content/drive/MyDrive/11785-project-medred/medred_ids_1000_fixed.csv")

# Directory for all eval outputs
EVAL_ROOT = Path(CFG.OUT_DIR) / "medred_eval_1000"
EVAL_ROOT.mkdir(parents=True, exist_ok=True)

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ============================================================
# 2) Helper functions (EM, confidence, Brier/ECE, ROUGE)
# ============================================================

def normalize_ws(s: str) -> str:
    return " ".join((s or "").split())

def normalize_simple(s: Optional[str]) -> str:
    """
    EM-style normalization (same as in earlier MedRed code).
    """
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def ws(text):
        return " ".join(text.split())
    txt = (s or "").lower()
    txt = remove_articles(txt)
    txt = remove_punc(txt)
    return ws(txt)

def EM(ans: str, key: str) -> bool:
    return normalize_simple(ans) == normalize_simple(key)

def extract_confidence(text: str) -> float:
    """
    Parse self-reported confidence from the model output.
    Looks for 'Confidence: p' or 'Confidence = p' (p in [0,1] or [0,100]).
    """
    if not text:
        return np.nan
    m = re.search(r"confidence\s*[:=]\s*([0-9]*\.?[0-9]+)", text, flags=re.IGNORECASE)
    if not m:
        return np.nan
    try:
        c = float(m.group(1))
        if c > 1.0 and c <= 100.0:
            c = c / 100.0  # interpret as percent
        c = max(0.0, min(1.0, c))
        return float(c)
    except Exception:
        return np.nan

def strip_confidence_lines(text: str) -> str:
    """
    Remove 'Confidence: ...' lines so ROUGE/readability are on answer text only.
    """
    if not text:
        return ""
    lines = text.splitlines()
    kept = [
        ln for ln in lines
        if not re.match(r"^\s*confidence\s*[:=]", ln, flags=re.IGNORECASE)
    ]
    return "\n".join(kept).strip()

def compute_brier(correct: np.ndarray, conf: np.ndarray) -> float:
    """
    Brier score for binary outcomes.
    correct: 0/1
    conf   : probabilities in [0,1], may contain NaN
    """
    conf = np.asarray(conf, dtype=float)
    correct = np.asarray(correct, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    return float(np.mean((p - y) ** 2))

def compute_ece(correct: np.ndarray, conf: np.ndarray, n_bins: int = 10) -> float:
    """
    Expected Calibration Error using scalar confidence.
    correct: 0/1
    conf   : in [0,1], may contain NaN
    """
    conf = np.asarray(conf, dtype=float)
    correct = np.asarray(correct, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    N = len(p)
    bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for b in range(n_bins):
        left = bin_edges[b]
        right = bin_edges[b + 1]
        if b < n_bins - 1:
            in_bin = (p >= left) & (p < right)
        else:
            in_bin = (p >= left) & (p <= right + 1e-8)
        if not np.any(in_bin):
            continue
        acc_b = float(y[in_bin].mean())
        conf_b = float(p[in_bin].mean())
        ece += (in_bin.sum() / N) * abs(acc_b - conf_b)
    return float(ece)

# ROUGE scorer (for multiprocessing)
def _init_scorer():
    global _SCORER
    _SCORER = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)

def _r1_pair(args):
    ref, hyp = args
    return _SCORER.score(ref, hyp)["rouge1"].fmeasure

# ============================================================
# 3) Fix the 1000 MedRed ids once, then pick 500-shard
# ============================================================

if ID_PATH_1000.exists():
    ids_1000 = pd.read_csv(ID_PATH_1000, dtype={"id": str})["id"].astype(str).tolist()
    print(f"[SAMPLING] Loaded fixed 1000 ids from {ID_PATH_1000}")
else:
    print(f"[SAMPLING] Creating new 1000-id subset and saving to {ID_PATH_1000}")
    df_sample = df.sample(n=1000, random_state=SEED).reset_index(drop=True)
    ids_1000 = df_sample["id"].astype(str).tolist()
    pd.DataFrame({"id": ids_1000}).to_csv(ID_PATH_1000, index=False)

assert len(ids_1000) == 1000, f"Expected 1000 ids, got {len(ids_1000)}"

SHARD_SIZE = 500
SHARD_INDEX = int(os.getenv("SHARD_INDEX", "1"))  # 0: first 500, 1: second 500

start = SHARD_INDEX * SHARD_SIZE
end = min(start + SHARD_SIZE, len(ids_1000))
ids_shard = ids_1000[start:end]
print(f"[SHARD] Using shard_index={SHARD_INDEX} with rows [{start}:{end}) → {len(ids_shard)} ids")

order_map = {id_: i for i, id_ in enumerate(ids_shard)}

df_shard = (
    df[df["id"].isin(ids_shard)]
    .assign(__order=lambda d: d["id"].map(order_map))
    .sort_values("__order")
    .drop(columns=["__order"])
    .reset_index(drop=True)
)

print("[SHARD] df_shard size:", len(df_shard))

# ============================================================
# 4) Batching/generation helpers for BASELINE (same as before)
# ============================================================

def make_batches_by_padded_cost(prompt_lens, budget_tokens, max_new, max_batch_samples, safety=0.9):
    batches, cur = [], []
    cur_max = 0
    eff = int(budget_tokens * safety)
    for i, L in enumerate(prompt_lens):
        new_max = max(cur_max, L)
        bs_next = len(cur) + 1
        cost = bs_next * (new_max + max_new)
        if cur and (cost > eff or len(cur) >= max_batch_samples):
            batches.append(cur)
            cur = [i]
            cur_max = L
        else:
            cur.append(i)
            cur_max = new_max
    if cur:
        batches.append(cur)
    return batches

def left_pad_collate(iid_list, device):
    maxL = max(len(x) for x in iid_list)
    pad_id = tok.pad_token_id
    input_ids = torch.full((len(iid_list), maxL), pad_id, dtype=torch.long)
    attn_mask = torch.zeros((len(iid_list), maxL), dtype=torch.bool)
    for r, ids_r in enumerate(iid_list):
        L = len(ids_r)
        input_ids[r, -L:] = torch.as_tensor(ids_r, dtype=torch.long)
        attn_mask[r,  -L:] = True
    return input_ids.to(device, non_blocking=True), attn_mask.to(device, non_blocking=True)

# ============================================================
# 5) BASELINE: original-style pipeline + confidence + calibration
# ============================================================

def build_baseline_prompt(q: str) -> str:
    """
    Original baseline prompt: system + user(question text).
    We now *add* a confidence line instruction, but otherwise keep the style.
    """
    user_content = (
        f"{q}\n\n"
        "After you provide your final answer, on a separate last line write "
        "'Confidence: p' where p is a number between 0 and 1 representing how "
        "confident you are that your answer is correct."
    )
    msgs = [
        {"role": "system", "content": CFG.SYSTEM_PROMPT},
        {"role": "user",   "content": user_content},
    ]
    try:
        return tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    except Exception:
        return (
            f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n{CFG.SYSTEM_PROMPT}\n"
            f"<|eot_id|><|start_header_id|>user<|end_header_id|>\n{user_content}\n"
            f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        )

def run_baseline_on_shard(df_in: pd.DataFrame):
    prompts = [build_baseline_prompt(q) for q in df_in["query"].tolist()]

    enc = tok(
        prompts,
        add_special_tokens=True,
        padding=False,
        truncation=True,
        max_length=CFG.MAX_INPUT_TOKENS,
        return_tensors=None,
    )
    input_ids_list = enc["input_ids"]
    lens = [len(x) for x in input_ids_list]

    order = np.argsort(lens).tolist()
    row_sorted = [i for i in order]
    iids_sorted = [input_ids_list[i] for i in order]
    lens_sorted = [lens[i] for i in order]

    batches = make_batches_by_padded_cost(
        lens_sorted,
        CFG.BATCH_TOKENS_BUDGET,
        CFG.MAX_NEW_TOKENS,
        CFG.MAX_BATCH_SAMPLES,
        safety=CFG.SAFETY,
    )

    eos_token_ids = [tok.eos_token_id]
    try:
        eot_id = tok.convert_tokens_to_ids("<|eot_id|>")
        if isinstance(eot_id, int) and eot_id != tok.eos_token_id:
            eos_token_ids.append(eot_id)
    except Exception:
        pass
    for stop_str in ["\n\n", "\n###", "\nUser:", "\nAssistant:"]:
        ids_tmp = tok(stop_str, add_special_tokens=False).input_ids
        if ids_tmp:
            eos_token_ids.append(ids_tmp[-1])

    pred_rows = []
    with torch.inference_mode():
        for b_ix, idxs in enumerate(tqdm(batches, desc=f"[baseline_shard{SHARD_INDEX}] generating")):
            batch_iids = [iids_sorted[i] for i in idxs]
            row_ids = [row_sorted[i] for i in idxs]

            input_ids, attn_mask = left_pad_collate(batch_iids, model.device)

            out = model.generate(
                input_ids=input_ids,
                attention_mask=attn_mask,
                max_new_tokens=CFG.MAX_NEW_TOKENS,
                do_sample=False,
                eos_token_id=eos_token_ids,
                pad_token_id=tok.pad_token_id,
                use_cache=True,
            )

            new_tokens = out[:, input_ids.shape[1]:]
            texts = tok.batch_decode(new_tokens, skip_special_tokens=True)

            for row_id, txt in zip(row_ids, texts):
                full_text = txt.strip()
                conf = extract_confidence(full_text)
                pred_clean = strip_confidence_lines(full_text)
                pred_rows.append({
                    "row_idx": row_id,
                    "prediction_raw": full_text,
                    "prediction": pred_clean,
                    "confidence": conf,
                })

            del input_ids, attn_mask, out, new_tokens, texts
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
            gc.collect()

    preds_df = pd.DataFrame(pred_rows).sort_values("row_idx").reset_index(drop=True)
    assert len(preds_df) == len(df_in)

    eval_df = df_in.reset_index(drop=True).copy()
    eval_df = pd.concat([eval_df, preds_df.drop(columns=["row_idx"])], axis=1)

    # Normalize for EM / ROUGE / readability
    eval_df["reference_clean"]  = eval_df["reference"].map(normalize_ws)
    eval_df["prediction_clean"] = eval_df["prediction"].fillna("").map(normalize_ws)

    # EM on full (clean) prediction vs reference (original baseline behavior)
    eval_df["em"] = [
        EM(pred, ref)
        for pred, ref in zip(eval_df["prediction_clean"], eval_df["reference_clean"])
    ]

    print(f"[baseline_shard{SHARD_INDEX}] eval rows:", len(eval_df))

    # ROUGE-1(F1)
    pairs = list(zip(eval_df["reference_clean"], eval_df["prediction_clean"]))
    with Pool(processes=max(1, cpu_count() - 1), initializer=_init_scorer) as pool:
        r1 = np.array(
            list(
                tqdm(
                    pool.imap(_r1_pair, pairs),
                    total=len(eval_df),
                    desc=f"[baseline_shard{SHARD_INDEX}] ROUGE-1(F1)",
                )
            ),
            dtype=np.float32,
        )
    eval_df["rouge1_f1"] = r1

    # Readability
    if textstat is not None:
        try:
            eval_df["readability_fre"]   = eval_df["prediction_clean"].map(textstat.flesch_reading_ease)
            eval_df["readability_fkgl"]  = eval_df["prediction_clean"].map(textstat.flesch_kincaid_grade)
            eval_df["readability_smog"]  = eval_df["prediction_clean"].map(textstat.smog_index)
        except Exception as e:
            print(f"WARNING: [baseline_shard{SHARD_INDEX}] readability computation failed:", e)
            eval_df["readability_fre"]   = np.nan
            eval_df["readability_fkgl"]  = np.nan
            eval_df["readability_smog"]  = np.nan
    else:
        eval_df["readability_fre"]   = np.nan
        eval_df["readability_fkgl"]  = np.nan
        eval_df["readability_smog"]  = np.nan

    # Calibration (ROUGE proxy + EM)
    r1_vals   = eval_df["rouge1_f1"].values.astype(float)
    em_vals   = eval_df["em"].astype(float).values
    conf_vals = eval_df["confidence"].values.astype(float)

    correct_proxy = np.where(
        np.isnan(r1_vals),
        0.0,
        (r1_vals >= ROUGE_TAU_FOR_ECE).astype(float),
    )

    brier_rouge = compute_brier(correct_proxy, conf_vals)
    ece_rouge   = compute_ece(correct_proxy, conf_vals, n_bins=ECE_N_BINS)

    brier_em = compute_brier(em_vals, conf_vals)
    ece_em   = compute_ece(em_vals, conf_vals, n_bins=ECE_N_BINS)

    summary = {
        "model": f"baseline_shard{SHARD_INDEX}",
        "n": int(len(eval_df)),
        "rouge1_f1_mean": float(np.nanmean(r1_vals)),
        "rouge1_f1_std":  float(np.nanstd(r1_vals)),
        "em_mean":        float(np.nanmean(em_vals)),
        "readability_fre_mean":   float(np.nanmean(eval_df["readability_fre"])),
        "readability_fkgl_mean":  float(np.nanmean(eval_df["readability_fkgl"])),
        "readability_smog_mean":  float(np.nanmean(eval_df["readability_smog"])),
        "brier_rouge_proxy": float(brier_rouge),
        "ece_rouge_proxy":   float(ece_rouge),
        "ece_rouge_tau":     float(ROUGE_TAU_FOR_ECE),
        "ece_n_bins":        int(ECE_N_BINS),
        "brier_em": float(brier_em),
        "ece_em":   float(ece_em),
    }

    return eval_df, summary

# ============================================================
# 6) BASELINE+CoT: original CoT-only agent + confidence + calibration
# ============================================================

# --- CoT instructions (same style as your CoT-only code, but with confidence line) ---
COT_INSTRUCTION = (
    "Solve a question answering task by having a Thought, then Finish with your answer. "
    "Thought can reason about the current situation. Finish[answer] returns the answer "
    "and finishes the task. You will be given a clinical note and question.\n"
    "After you give your final answer, on a new line write: 'Confidence: p' where p is a "
    "number between 0 and 1 indicating how confident you are that your answer is correct.\n"
)

COT_FEWSHOT = """Relevant Context: The Nile River is the longest river in the world, spanning approximately 6,650 kilometers (4,132 miles) in length. It flows through eleven countries in northeastern Africa, including Egypt, Sudan, and Uganda.
Question: What is the longest river in the world?
Thought: The question asks for the longest river in the world, which I know is the Nile River based on the context provided.
Action: Finish[Nile River]

Relevant Context: Ludwig van Beethoven was a German composer and pianist who was a crucial figure in the transition between the Classical and Romantic eras in Western classical music. One of his most famous compositions is the Symphony No. 9, also known as the "Choral" symphony.
Question: Which composer created Symphony No. 9?
Thought: The question is asking for the composer of Symphony No. 9. Based on the context, I know that Ludwig van Beethoven composed this symphony.
Action: Finish[Ludwig van Beethoven]

Relevant Context: Photosynthesis is the process by which green plants and some other organisms convert light energy into chemical energy. During this process, plants absorb sunlight, carbon dioxide, and water to produce glucose and oxygen.
Question: What do plants produce during photosynthesis?
Thought: The question is asking about the products of photosynthesis. From the context, I know that plants produce glucose and oxygen during this process.
Action: Finish[Glucose and Oxygen]
"""

# --- Local LLM wrapper (same style as in your Reflexion code) ---
class LocalNotebookLLM:
    """
    Simple wrapper: llm(prompt) -> text, with max_new_tokens and stop tokens.
    Assumes `tok` and `model` are already loaded and on GPU/CPU.
    """
    def __init__(
        self,
        tokenizer,
        model,
        device: Optional[torch.device] = None,
        max_new_tokens: int = 160,
        temperature: float = 0.0,
        stop_tokens: Optional[List[str]] = None,
    ):
        self.tok = tokenizer
        self.model = model
        self.device = device or getattr(
            model,
            "device",
            torch.device("cuda" if torch.cuda.is_available() else "cpu")
        )
        self.max_new_tokens = int(max_new_tokens)
        self.temperature = float(temperature)
        self.stop_tokens = stop_tokens or [
            "\n###", "\nUser:", "\nAssistant:", "\nObservation", "Observation"
        ]

    def _apply_stop(self, text: str, stops: Optional[List[str]] = None) -> str:
        stops = stops if stops is not None else self.stop_tokens
        if not stops:
            return text
        idx = None
        for s in stops:
            if not s:
                continue
            p = text.find(s)
            if p > 0 and (idx is None or p < idx):
                idx = p
        return text[:idx] if idx is not None else text

    def __call__(self, prompt: str, max_new_tokens: Optional[int] = None) -> str:
        mnt = int(max_new_tokens) if max_new_tokens is not None else self.max_new_tokens
        max_pos = getattr(self.model.config, "max_position_embeddings", 2048)
        allowed_input = max(64, max_pos - mnt - 32)

        enc = self.tok(
            prompt,
            truncation=True,
            max_length=allowed_input,
            return_tensors="pt",
        )
        input_ids = enc["input_ids"].to(self.device)
        attn_mask = enc.get("attention_mask", None)
        if attn_mask is not None:
            attn_mask = attn_mask.to(self.device)

        with torch.inference_mode():
            out = self.model.generate(
                input_ids=input_ids,
                attention_mask=attn_mask,
                max_new_tokens=mnt,
                do_sample=(self.temperature > 0.0),
                temperature=self.temperature,
                pad_token_id=self.tok.pad_token_id,
                eos_token_id=self.tok.eos_token_id,
                use_cache=True,
            )

        gen = out[0, input_ids.shape[1]:]
        text = self.tok.decode(gen, skip_special_tokens=True)
        return self._apply_stop(text).strip()

llm_cot = LocalNotebookLLM(
    tok,
    model,
    device=model.device,
    max_new_tokens=max(CFG.MAX_NEW_TOKENS, 160),
)
print("[CoT-only] LLM ready on", model.device)

# --- Answer & rationale extractors (same logic as before, but confidence-aware) ---
def extract_medred_answer(text: str) -> str:
    """
    Short answer for EM.
    """
    if not text:
        return ""
    # 1) Action: Finish[answer]
    m = re.search(r"Action:\s*Finish\[(.*?)\]", text, flags=re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1).strip()
    # 2) Any Finish[answer]
    m = re.search(r"Finish\[(.*?)\]", text, flags=re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1).strip()
    # 3) "Final answer: ..."
    m = re.search(r"final answer\s*:\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        line = m.group(1).strip().splitlines()[0]
        return line.strip()
    # 4) fallback: last non-empty line not starting with Thought/Action/Confidence
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    for line in reversed(lines):
        if re.match(r"^(thought|action|confidence)\b", line, flags=re.IGNORECASE):
            continue
        return line
    return text.strip()

def extract_rationale_last_paragraph(text: str) -> str:
    """
    Long rationale: last non-empty paragraph of the trajectory,
    ignoring pure 'Confidence: ...' paragraphs.
    """
    if not text:
        return ""
    paras = [p.strip() for p in text.split("\n\n") if p.strip()]
    if not paras:
        return text.strip()
    non_conf = [p for p in paras if not re.match(r"^confidence\b", p, flags=re.IGNORECASE)]
    if non_conf:
        return non_conf[-1]
    return paras[-1]

@dataclass
class MedRedCoTOnlyAgent:
    ex_id: str
    query: str           # full MedRed text (title+body + question)
    answer_key: str
    llm: LocalNotebookLLM
    max_steps: int = 1   # CoT-only single attempt

    # each attempt: {trajectory, answer, rationale, confidence, correct}
    attempts: List[Dict[str, Any]] = field(default_factory=list)

    def is_correct(self) -> bool:
        return bool(self.attempts and self.attempts[-1]["correct"])

    def _build_agent_prompt(self) -> str:
        prompt = COT_INSTRUCTION
        prompt += "\nHere are some examples:\n"
        prompt += COT_FEWSHOT
        prompt += "\n(END OF EXAMPLES)\n"

        prompt += (
            "\nRelevant Context (clinical note + question):\n"
            f"{self.query.strip()}\n"
            "Question: Based on the above, what is the best answer for the patient?\n"
            "Thought 1:"
        )
        return prompt

    def step(self) -> Dict[str, Any]:
        prompt = self._build_agent_prompt()
        traj = self.llm(prompt, max_new_tokens=CFG.MAX_NEW_TOKENS + 80)

        ans       = extract_medred_answer(traj)                  # short EM answer
        rationale = extract_rationale_last_paragraph(traj)       # long rationale
        conf      = extract_confidence(traj)                     # self-reported p(correct)

        correct = EM(ans, self.answer_key)
        rec = {
            "trajectory": traj,
            "answer": ans,
            "rationale": rationale,
            "confidence": conf,
            "correct": bool(correct),
        }
        self.attempts.append(rec)
        return rec

def run_cot_on_shard(df_in: pd.DataFrame):
    """
    Original CoT-only pipeline:
    - One CoT attempt per example
    - EM on short answer (Finish[...] etc.)
    - ROUGE & readability on long rationale (rationaleK)
    - Adds calibration metrics via confidence.
    """
    results = []

    for ex_idx, row in enumerate(
        tqdm(
            df_in[["id", "query", "reference"]].itertuples(index=False),
            total=len(df_in),
            desc=f"[baseline_cot_shard{SHARD_INDEX}] CoT-only episodes",
        ),
        start=1,
    ):
        ex_id     = row.id
        query     = row.query
        reference = row.reference

        agent = MedRedCoTOnlyAgent(
            ex_id=ex_id,
            query=query,
            answer_key=reference,
            llm=llm_cot,
            max_steps=1,
        )

        first_rec   = agent.step()
        pred1       = first_rec["answer"] or ""
        rationale1  = first_rec["rationale"] or ""
        conf1       = first_rec["confidence"]
        correct1    = bool(first_rec["correct"])

        # For CoT-only, "final" == first attempt
        predK      = pred1
        rationaleK = rationale1
        confK      = conf1
        correctK   = correct1

        results.append({
            "id": ex_id,
            "short_answer": predK,      # EM target
            "rationale": rationaleK,    # long prediction for ROUGE/readability
            "reference": reference,
            "confidence": confK,
            "em": correctK,
            "trajectory": first_rec["trajectory"],
        })

    df_cot = pd.DataFrame(results)

    # Build eval df: use rationale as "prediction"
    eval_df = df_cot.copy()
    eval_df["reference_clean"]  = eval_df["reference"].map(normalize_ws)
    eval_df["prediction"]       = eval_df["rationale"].fillna("")
    eval_df["prediction_clean"] = eval_df["prediction"].map(normalize_ws)

    print(f"[baseline_cot_shard{SHARD_INDEX}] eval rows:", len(eval_df))

    # ROUGE-1(F1) on rationaleK vs reference (original CoT-only metric behavior)
    pairs = list(zip(eval_df["reference_clean"], eval_df["prediction_clean"]))
    with Pool(processes=max(1, cpu_count() - 1), initializer=_init_scorer) as pool:
        r1 = np.array(
            list(
                tqdm(
                    pool.imap(_r1_pair, pairs),
                    total=len(eval_df),
                    desc=f"[baseline_cot_shard{SHARD_INDEX}] ROUGE-1(F1) on rationale",
                )
            ),
            dtype=np.float32,
        )
    eval_df["rouge1_f1"] = r1

    # Readability on rationaleK
    if textstat is not None:
        try:
            eval_df["readability_fre"]   = eval_df["prediction_clean"].map(textstat.flesch_reading_ease)
            eval_df["readability_fkgl"]  = eval_df["prediction_clean"].map(textstat.flesch_kincaid_grade)
            eval_df["readability_smog"]  = eval_df["prediction_clean"].map(textstat.smog_index)
        except Exception as e:
            print(f"WARNING: [baseline_cot_shard{SHARD_INDEX}] readability computation failed:", e)
            eval_df["readability_fre"]   = np.nan
            eval_df["readability_fkgl"]  = np.nan
            eval_df["readability_smog"]  = np.nan
    else:
        eval_df["readability_fre"]   = np.nan
        eval_df["readability_fkgl"]  = np.nan
        eval_df["readability_smog"]  = np.nan

    # Calibration (ROUGE proxy + EM) using self-reported confidence
    r1_vals   = eval_df["rouge1_f1"].values.astype(float)
    em_vals   = eval_df["em"].astype(float).values  # EM on short_answer
    conf_vals = eval_df["confidence"].values.astype(float)

    correct_proxy = np.where(
        np.isnan(r1_vals),
        0.0,
        (r1_vals >= ROUGE_TAU_FOR_ECE).astype(float),
    )

    brier_rouge = compute_brier(correct_proxy, conf_vals)
    ece_rouge   = compute_ece(correct_proxy, conf_vals, n_bins=ECE_N_BINS)

    brier_em = compute_brier(em_vals, conf_vals)
    ece_em   = compute_ece(em_vals, conf_vals, n_bins=ECE_N_BINS)

    summary = {
        "model": f"baseline_cot_shard{SHARD_INDEX}",
        "n": int(len(eval_df)),
        "rouge1_f1_mean": float(np.nanmean(r1_vals)),
        "rouge1_f1_std":  float(np.nanstd(r1_vals)),
        "em_mean":        float(np.nanmean(em_vals)),
        "readability_fre_mean":   float(np.nanmean(eval_df["readability_fre"])),
        "readability_fkgl_mean":  float(np.nanmean(eval_df["readability_fkgl"])),
        "readability_smog_mean":  float(np.nanmean(eval_df["readability_smog"])),
        "brier_rouge_proxy": float(brier_rouge),
        "ece_rouge_proxy":   float(ece_rouge),
        "ece_rouge_tau":     float(ROUGE_TAU_FOR_ECE),
        "ece_n_bins":        int(ECE_N_BINS),
        "brier_em": float(brier_em),
        "ece_em":   float(ece_em),
    }

    return eval_df, summary

# ============================================================
# 7) Run BASELINE+CoT and BASELINE on this shard and save
# ============================================================

# --- Baseline + CoT ---
eval_cot, summary_cot = run_cot_on_shard(df_shard)
cot_dir = EVAL_ROOT / f"baseline_cot_shard{SHARD_INDEX}"
cot_dir.mkdir(parents=True, exist_ok=True)
eval_cot.to_csv(cot_dir / "per_example_scores.csv", index=False)
with open(cot_dir / "summary.json", "w") as f:
    json.dump(summary_cot, f, indent=2)
print("\n[BASELINE+CoT] Summary:")
print(json.dumps(summary_cot, indent=2))


### Combine shard 0,1 for baseline and basline+CoT

In [ ]:
import pandas as pd
import numpy as np
import json

# --- helper functions (same as in your training code) ---
def compute_brier(correct: np.ndarray, conf: np.ndarray) -> float:
    correct = np.asarray(correct, dtype=float)
    conf = np.asarray(conf, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    return float(np.mean((p - y) ** 2))

def compute_ece(correct: np.ndarray, conf: np.ndarray, n_bins: int = 10) -> float:
    correct = np.asarray(correct, dtype=float)
    conf = np.asarray(conf, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    N = len(p)
    bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for b in range(n_bins):
        left = bin_edges[b]
        right = bin_edges[b + 1]
        if b < n_bins - 1:
            in_bin = (p >= left) & (p < right)
        else:
            in_bin = (p >= left) & (p <= right + 1e-8)
        if not np.any(in_bin):
            continue
        acc_b = float(y[in_bin].mean())
        conf_b = float(p[in_bin].mean())
        ece += (in_bin.sum() / N) * abs(acc_b - conf_b)
    return float(ece)

# ---------- 1) Load shards and concat ----------
path_base0 = "/kaggle/input/baseline/baseline_shard0.csv"
path_base1 = "/kaggle/input/baseline/baseline_shard1.csv"

df_base0 = pd.read_csv(path_base0)
df_base1 = pd.read_csv(path_base1)

df_base_all = pd.concat([df_base0, df_base1], ignore_index=True)

# sanity check
df_base_all["id"] = df_base_all["id"].astype(str)
print("Baseline combined rows:", len(df_base_all))
print("Unique ids:", df_base_all["id"].nunique())

# ---------- 2) Compute combined metrics ----------
r1_vals   = df_base_all["rouge1_f1"].values.astype(float)
em_vals   = df_base_all["em"].astype(float).values
conf_vals = df_base_all["confidence"].astype(float).values

# if you used ROUGE_TAU_FOR_ECE=0.2 and ECE_N_BINS=10:
ROUGE_TAU_FOR_ECE = 0.2
ECE_N_BINS = 10

correct_proxy = np.where(
    np.isnan(r1_vals),
    0.0,
    (r1_vals >= ROUGE_TAU_FOR_ECE).astype(float),
)

brier_rouge = compute_brier(correct_proxy, conf_vals)
ece_rouge   = compute_ece(correct_proxy, conf_vals, n_bins=ECE_N_BINS)
brier_em    = compute_brier(em_vals, conf_vals)
ece_em      = compute_ece(em_vals, conf_vals, n_bins=ECE_N_BINS)

summary_base_all = {
    "model": "baseline_all_1000",
    "n": int(len(df_base_all)),
    "rouge1_f1_mean": float(np.nanmean(r1_vals)),
    "rouge1_f1_std":  float(np.nanstd(r1_vals)),
    "em_mean":        float(np.nanmean(em_vals)),
    "readability_fre_mean":   float(np.nanmean(df_base_all["readability_fre"])),
    "readability_fkgl_mean":  float(np.nanmean(df_base_all["readability_fkgl"])),
    "readability_smog_mean":  float(np.nanmean(df_base_all["readability_smog"])),
    "brier_rouge_proxy": float(brier_rouge),
    "ece_rouge_proxy":   float(ece_rouge),
    "ece_rouge_tau":     float(ROUGE_TAU_FOR_ECE),
    "ece_n_bins":        int(ECE_N_BINS),
    "brier_em": float(brier_em),
    "ece_em":   float(ece_em),
}

print(json.dumps(summary_base_all, indent=2))

# Optional: save combined per-example + summary
out_dir = "/kaggle/working/medred_baseline_all"
import os
os.makedirs(out_dir, exist_ok=True)
df_base_all.to_csv(f"{out_dir}/per_example_scores.csv", index=False)
with open(f"{out_dir}/summary.json", "w") as f:
    json.dump(summary_base_all, f, indent=2)
print("Saved combined baseline outputs to", out_dir)


import pandas as pd
import numpy as np
import json

# re-use compute_brier / compute_ece from above

path_cot0 = "/kaggle/input/baseline/baseline_cot_shard0.csv"
path_cot1 = "/kaggle/input/baseline/baseline_cot_shard1.csv"

df_cot0 = pd.read_csv(path_cot0)
df_cot1 = pd.read_csv(path_cot1)

df_cot_all = pd.concat([df_cot0, df_cot1], ignore_index=True)
df_cot_all["id"] = df_cot_all["id"].astype(str)

print("Baseline+CoT combined rows:", len(df_cot_all))
print("Unique ids:", df_cot_all["id"].nunique())

r1_vals   = df_cot_all["rouge1_f1"].values.astype(float)
em_vals   = df_cot_all["em"].astype(float).values
conf_vals = df_cot_all["confidence"].astype(float).values

ROUGE_TAU_FOR_ECE = 0.2
ECE_N_BINS = 10

correct_proxy = np.where(
    np.isnan(r1_vals),
    0.0,
    (r1_vals >= ROUGE_TAU_FOR_ECE).astype(float),
)

brier_rouge = compute_brier(correct_proxy, conf_vals)
ece_rouge   = compute_ece(correct_proxy, conf_vals, n_bins=ECE_N_BINS)
brier_em    = compute_brier(em_vals, conf_vals)
ece_em      = compute_ece(em_vals, conf_vals, n_bins=ECE_N_BINS)

summary_cot_all = {
    "model": "baseline_cot_all_1000",
    "n": int(len(df_cot_all)),
    "rouge1_f1_mean": float(np.nanmean(r1_vals)),
    "rouge1_f1_std":  float(np.nanstd(r1_vals)),
    "em_mean":        float(np.nanmean(em_vals)),
    "readability_fre_mean":   float(np.nanmean(df_cot_all["readability_fre"])),
    "readability_fkgl_mean":  float(np.nanmean(df_cot_all["readability_fkgl"])),
    "readability_smog_mean":  float(np.nanmean(df_cot_all["readability_smog"])),
    "brier_rouge_proxy": float(brier_rouge),
    "ece_rouge_proxy":   float(ece_rouge),
    "ece_rouge_tau":     float(ROUGE_TAU_FOR_ECE),
    "ece_n_bins":        int(ECE_N_BINS),
    "brier_em": float(brier_em),
    "ece_em":   float(ece_em),
}

print(json.dumps(summary_cot_all, indent=2))

out_dir_cot = "/kaggle/working/medred_baseline_cot_all"
import os
os.makedirs(out_dir_cot, exist_ok=True)
df_cot_all.to_csv(f"{out_dir_cot}/per_example_scores.csv", index=False)
with open(f"{out_dir_cot}/summary.json", "w") as f:
    json.dump(summary_cot_all, f, indent=2)
print("Saved combined baseline+CoT outputs to", out_dir_cot)


## Reflexion V1 (short-form)

In [8]:
# ============================================================
#  HELPERS: EM, confidence, calibration
# ============================================================
def normalize_simple(s: Optional[str]) -> str:
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def ws(text):
        return " ".join(text.split())
    txt = (s or "").lower()
    txt = remove_articles(txt)
    txt = remove_punc(txt)
    return ws(txt)

def EM(ans: str, key: str) -> bool:
    return normalize_simple(ans) == normalize_simple(key)

def extract_confidence(text: str) -> float:
    """
    Parse self-reported confidence from model output.
    Looks for 'Confidence: p' or 'Confidence = p' with p in [0,1] or [0,100].
    """
    if not text:
        return np.nan
    m = re.search(r"confidence\s*[:=]\s*([0-9]*\.?[0-9]+)", text, flags=re.IGNORECASE)
    if not m:
        return np.nan
    try:
        c = float(m.group(1))
        if c > 1.0 and c <= 100.0:
            c = c / 100.0
        c = max(0.0, min(1.0, c))
        return float(c)
    except Exception:
        return np.nan

def compute_brier(correct: np.ndarray, conf: np.ndarray) -> float:
    """
    Brier score for binary outcomes.
    correct: array of 0/1
    conf   : array of probabilities in [0,1] (may contain NaN)
    """
    correct = np.asarray(correct, dtype=float)
    conf = np.asarray(conf, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    return float(np.mean((p - y) ** 2))

def compute_ece(correct: np.ndarray, conf: np.ndarray, n_bins: int = 10) -> float:
    """
    Expected Calibration Error (ECE) with scalar confidence.
    correct: array of 0/1
    conf   : array of probabilities in [0,1] (may contain NaN)
    """
    correct = np.asarray(correct, dtype=float)
    conf = np.asarray(conf, dtype=float)
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask]
    p = conf[mask]
    N = len(p)
    bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for b in range(n_bins):
        left = bin_edges[b]
        right = bin_edges[b + 1]
        if b < n_bins - 1:
            in_bin = (p >= left) & (p < right)
        else:
            in_bin = (p >= left) & (p <= right + 1e-8)
        if not np.any(in_bin):
            continue
        acc = np.mean(y[in_bin])
        conf_avg = np.mean(p[in_bin])
        ece += (in_bin.sum() / N) * abs(acc - conf_avg)
    return float(ece)

def normalize_ws(s: str) -> str:
    return " ".join((s or "").split())

In [ ]:
# ============================================================
#  LOCAL LLM WRAPPER
# ============================================================
class LocalNotebookLLM:
    """
    Simple wrapper: llm(prompt) -> text, with max_new_tokens and stop tokens.
    """
    def __init__(
        self,
        tokenizer,
        model,
        device: Optional[torch.device] = None,
        max_new_tokens: int = 160,
        temperature: float = 0.0,
        stop_tokens: Optional[List[str]] = None,
    ):
        self.tok = tokenizer
        self.model = model
        self.device = device or getattr(
            model,
            "device",
            torch.device("cuda" if torch.cuda.is_available() else "cpu")
        )
        self.max_new_tokens = int(max_new_tokens)
        self.temperature = float(temperature)
        self.stop_tokens = stop_tokens or [
            "\n###", "\nUser:", "\nAssistant:", "\nObservation", "Observation"
        ]

    def _apply_stop(self, text: str, stops: Optional[List[str]] = None) -> str:
        stops = stops if stops is not None else self.stop_tokens
        if not stops:
            return text
        idx = None
        for s in stops:
            if not s:
                continue
            p = text.find(s)
            if p > 0 and (idx is None or p < idx):
                idx = p
        return text[:idx] if idx is not None else text

    def __call__(self, prompt: str, max_new_tokens: Optional[int] = None) -> str:
        mnt = int(max_new_tokens) if max_new_tokens is not None else self.max_new_tokens
        max_pos = getattr(self.model.config, "max_position_embeddings", 2048)
        allowed_input = max(64, max_pos - mnt - 32)

        enc = self.tok(
            prompt,
            truncation=True,
            max_length=allowed_input,
            return_tensors="pt",
        )
        input_ids = enc["input_ids"].to(self.device)
        attn_mask = enc.get("attention_mask", None)
        if attn_mask is not None:
            attn_mask = attn_mask.to(self.device)

        with torch.inference_mode():
            out = self.model.generate(
                input_ids=input_ids,
                attention_mask=attn_mask,
                max_new_tokens=mnt,
                do_sample=(self.temperature > 0.0),
                temperature=self.temperature,
                pad_token_id=self.tok.pad_token_id,
                eos_token_id=self.tok.eos_token_id,
                use_cache=True,
            )

        gen = out[0, input_ids.shape[1]:]
        text = self.tok.decode(gen, skip_special_tokens=True)
        return self._apply_stop(text).strip()

llm = LocalNotebookLLM(
    tok,
    model,
    device=DEVICE,
    max_new_tokens=max(CFG.MAX_NEW_TOKENS, 160),
)
print("Reflexion LLM ready on", DEVICE)

# ============================================================
#  REFLEXION INSTRUCTIONS & EXTRACTORS
# ============================================================
class ReflexionStrategy(Enum):
    NONE = "none"
    LAST_ATTEMPT = "last_attempt"
    REFLEXION = "reflexion"
    LAST_ATTEMPT_AND_REFLEXION = "last_attempt_and_reflexion"

COT_INSTRUCTION = (
    "Solve a question answering task by having a Thought, then Finish with your answer. "
    "Thought can reason about the current situation. Finish[answer] returns the answer "
    "and finishes the task. You will be given a clinical note and question.\n"
    "After you provide your final answer, on a new last line write 'Confidence: p' where "
    "p is a number between 0 and 1 indicating how confident you are that your answer is correct.\n"
)

REFLECT_INSTRUCTION = (
    "You are an advanced reasoning agent that can improve based on self-reflection. "
    "You will be given a previous reasoning trial on a clinical QA task in which your "
    "answer was judged incorrect or low-quality. In a few sentences:\n"
    "1. Briefly explain why your reasoning or answer may have been wrong or incomplete.\n"
    "2. State 1–2 concrete rules you will follow next time to avoid the same mistake.\n"
)

COT_FEWSHOT = """Relevant Context: The Nile River is the longest river in the world, spanning approximately 6,650 kilometers (4,132 miles) in length. It flows through eleven countries in northeastern Africa, including Egypt, Sudan, and Uganda.
Question: What is the longest river in the world?
Thought: The question asks for the longest river in the world, which I know is the Nile River based on the context provided.
Action: Finish[Nile River]

Relevant Context: Ludwig van Beethoven was a German composer and pianist who was a crucial figure in the transition between the Classical and Romantic eras in Western classical music. One of his most famous compositions is the Symphony No. 9, also known as the "Choral" symphony.
Question: Which composer created Symphony No. 9?
Thought: The question is asking for the composer of Symphony No. 9. Based on the context, I know that Ludwig van Beethoven composed this symphony.
Action: Finish[Ludwig van Beethoven]

Relevant Context: Photosynthesis is the process by which green plants and some other organisms convert light energy into chemical energy. During this process, plants absorb sunlight, carbon dioxide, and water to produce glucose and oxygen.
Question: What do plants produce during photosynthesis?
Thought: The question is asking about the products of photosynthesis. From the context, I know that plants produce glucose and oxygen during this process.
Action: Finish[Glucose and Oxygen]
"""

COT_REFLECT_FEWSHOT = """
Relevant Context: Ernest Hemingway's novel "The Old Man and the Sea" tells the story of Santiago, an aging Cuban fisherman, who struggles to catch a giant marlin in the Gulf Stream. The book won the Pulitzer Prize for Fiction in 1953 and contributed to Hemingway's Nobel Prize for Literature in 1954.
Question: Which literary award did "The Old Man and the Sea" contribute to Hemingway winning?
Thought: The question is asking which award "The Old Man and the Sea" contributed to Hemingway winning. Based on the context, I know the novel won the Pulitzer Prize for Fiction and contributed to his Nobel Prize for Literature.
Action: Finish[Pulitzer Prize for Fiction]

Reflection: My answer was correct based on the context, but may not be the exact answer stored by the grading environment. Next time, I should try to provide a less verbose answer like "Pulitzer Prize" or "Nobel Prize."

Context: On 14 October 1947, Chuck Yeager, a United States Air Force test pilot, became the first person to break the sound barrier by flying the Bell X-1 experimental aircraft at an altitude of 45,000 feet.
Charles Elwood "Chuck" Yeager (13 February 1923 - 7 December 2020) was a United States Air Force officer, flying ace, and test pilot. He is best known for becoming the first person to break the sound barrier, which he achieved in the Bell X-1 aircraft named Glamorous Glennis. Yeager was also a distinguished fighter pilot during World War II and was credited with shooting down at least 12 enemy aircraft. In 1973, he was inducted into the National Aviation Hall of Fame for his significant contributions to aviation.
Question: Who is the first person to break the sound barrier?
Thought: The question is asking for the first person to break the sound barrier. From the context, I know that Chuck Yeager, a United States Air Force test pilot, was the first person to break the sound barrier.
Action: Finish[Chuck Yeager]

Reflection: Upon reflecting on the incorrect answer I provided, I realize that I may not have given the full name of the individual in question. In the context, both the given name and the nickname were mentioned, and I only used the nickname in my response. This could have been the reason my answer was deemed incorrect. Moving forward, when attempting this question again or similar questions, I will make sure to include the complete name of the person, which consists of their given name, any middle names, and their nickname (if applicable). This will help ensure that my answer is more accurate and comprehensive.
"""

def extract_medred_answer(text: str) -> str:
    """
    Robust extractor for the short MedRed answer.
    """
    if not text:
        return ""
    # 1) Action: Finish[answer]
    m = re.search(r"Action:\s*Finish\[(.*?)\]", text, flags=re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1).strip()
    # 2) Any Finish[answer]
    m = re.search(r"Finish\[(.*?)\]", text, flags=re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1).strip()
    # 3) "Final answer: ..."
    m = re.search(r"final answer\s*:\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        line = m.group(1).strip().splitlines()[0]
        return line.strip()
    # 4) Fallback: last non-empty line not starting with Thought/Action/Confidence
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    for line in reversed(lines):
        if re.match(r"^(thought|action|confidence)\b", line, flags=re.IGNORECASE):
            continue
        return line
    return text.strip()

@dataclass
class MedRedCoTAgent:
    ex_id: str
    query: str
    answer_key: str
    llm: LocalNotebookLLM
    max_steps: int = MAX_STEPS
    reflexion_strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

    attempts: List[Dict[str, Any]] = field(default_factory=list)
    reflections: List[str] = field(default_factory=list)

    def is_correct(self) -> bool:
        return bool(self.attempts and self.attempts[-1]["correct"])

    def _format_reflections_for_prompt(self) -> str:
        if not self.reflections:
            return ""
        header = (
            "You have attempted this question before and failed. "
            "The following reflection(s) give a plan to avoid failing in the same way.\n"
            "Reflections:\n"
        )
        bullets = "\n".join(f"- {r.strip()}" for r in self.reflections)
        return header + bullets

    def _build_agent_prompt(self) -> str:
        prompt = COT_INSTRUCTION
        prompt += "\nHere are some examples:\n"
        prompt += COT_FEWSHOT
        prompt += "\n(END OF EXAMPLES)\n"

        if self.reflexion_strategy in (
            ReflexionStrategy.REFLEXION,
            ReflexionStrategy.LAST_ATTEMPT_AND_REFLEXION
        ) and self.reflections:
            prompt += "\n" + self._format_reflections_for_prompt() + "\n"

        prompt += (
            "\nRelevant Context (clinical note + question):\n"
            f"{self.query.strip()}\n"
            "Question: Based on the above, what is the best answer for the patient?\n"
            "Thought 1:"
        )
        return prompt

    def step(self) -> Dict[str, Any]:
        prompt = self._build_agent_prompt()
        traj = self.llm(prompt, max_new_tokens=CFG.MAX_NEW_TOKENS + 80)
        ans = extract_medred_answer(traj)
        conf = extract_confidence(traj)
        correct = EM(ans, self.answer_key)
        rec = {
            "trajectory": traj,
            "answer": ans,
            "confidence": conf,
            "correct": bool(correct),
        }
        self.attempts.append(rec)
        return rec

    def reflect(self) -> Optional[str]:
        if not self.attempts:
            return None
        last = self.attempts[-1]
        refl_prompt = (
            REFLECT_INSTRUCTION
            + "\n\nHere are some reflection examples:\n"
            + COT_REFLECT_FEWSHOT
            + "\n(END OF EXAMPLES)\n"
            + "\nClinical case and question:\n"
            + self.query.strip()
            + "\n\nYour previous reasoning and answer:\n"
            + last["trajectory"]
            + "\n\nReflection:\n"
        )
        refl_text = self.llm(refl_prompt, max_new_tokens=160)
        self.reflections.append(refl_text.strip())
        return refl_text

# ============================================================
#  RUN REFLEXION EPISODES ON THIS SHARD (~250 IDS)
# ============================================================
results_reflex = []

for ex_idx, (_, row) in enumerate(
    tqdm(
        df_reflex_in[["id", "query", "reference"]].iterrows(),
        total=len(df_reflex_in),
        desc=f"Reflexion episodes (shard={CFG.SHARD_INDEX})",
    ),
    start=1,
):
    agent = MedRedCoTAgent(
        ex_id=row["id"],
        query=row["query"],
        answer_key=row["reference"],
        llm=llm,
        max_steps=MAX_STEPS,
        reflexion_strategy=ReflexionStrategy.REFLEXION,
    )

    # First attempt
    first_rec   = agent.step()
    pred1       = first_rec["answer"] or ""
    conf1       = first_rec["confidence"]
    correct1    = bool(first_rec["correct"])

    # Reflexion loop
    refl_count = 0
    while (not agent.is_correct()) and len(agent.attempts) < MAX_STEPS and refl_count < MAX_REFLECT_ATTEMPTS:
        agent.reflect()
        refl_count += 1
        agent.step()

    # Final attempt
    final_rec   = agent.attempts[-1]
    predK       = final_rec["answer"] or ""
    confK       = final_rec["confidence"]
    correctK    = bool(final_rec["correct"])

    results_reflex.append({
        "id": row["id"],
        "pred1": pred1,
        "predK": predK,
        "reference": row["reference"],
        "correct1": correct1,
        "correctK": correctK,
        "confidence1": conf1,
        "confidenceK": confK,
        "n_attempts": len(agent.attempts),
        "n_reflections": refl_count,
        "last_reflection": agent.reflections[-1] if agent.reflections else "",
    })

df_refl = pd.DataFrame(results_reflex)

out_name = f"medred_reflexion_v1_shard{CFG.SHARD_INDEX}_N{len(df_reflex_in)}.csv"
refl_path = CFG.OUT_DIR / out_name
df_refl.to_csv(refl_path, index=False)
print(f"\nSaved Reflexion v1 shard outputs to {refl_path}")

# Sanity check
missing_mask = df_refl["predK"].astype(str).str.strip() == ""
print("Missing / empty predK:", missing_mask.sum())
if missing_mask.sum() > 0:
    print(df_refl[missing_mask].head())

# ---- EM summary ----
em1 = df_refl["correct1"].mean()
emK = df_refl["correctK"].mean()
improved = ((~df_refl["correct1"]) & df_refl["correctK"]).sum()
worsened = (df_refl["correct1"] & (~df_refl["correctK"])).sum()
unchanged = len(df_refl) - improved - worsened

print("\n=== Reflexion EM summary on shard (CoT-style) ===")
print(f"n examples           : {len(df_refl)}")
print(f"EM step1 (pred1)     : {em1:.4f}")
print(f"EM final (predK)     : {emK:.4f}")
print(f"Δ EM (final - step1) : {emK - em1:+.4f}")
print(f"Improved (False→True): {improved}")
print(f"Worsened (True→False): {worsened}")
print(f"Unchanged            : {unchanged}")

# ============================================================
#  REFLEXION METRICS: ROUGE + MoverScore + READABILITY + BRIER/ECE
# ============================================================
def _init_scorer():
    global _SCORER
    _SCORER = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)

def _r1_pair(args):
    ref, hyp = args
    return _SCORER.score(ref, hyp)["rouge1"].fmeasure

def normalize(s: str) -> str:
    return " ".join((s or "").split())

reflex_ids = df_refl["id"].astype(str).unique()

df_sub = df[df["id"].astype(str).isin(reflex_ids)][["id", "reference"]].copy()
df_sub["id"] = df_sub["id"].astype(str)

# prediction, correctness, and confidence
refl_preds = (
    df_refl[["id", "predK", "correctK", "confidenceK"]]
    .rename(columns={"predK": "prediction", "confidenceK": "confidence"})
    .astype({"id": str})
)

eval_df_refl = df_sub.merge(refl_preds, on="id", how="left")
eval_df_refl["reference"]  = eval_df_refl["reference"].map(normalize)
eval_df_refl["prediction"] = eval_df_refl["prediction"].fillna("").map(normalize)

print("\nReflexion eval rows (should equal shard size):", len(eval_df_refl))

# ---- ROUGE-1(F1) ----
pairs = list(zip(eval_df_refl["reference"], eval_df_refl["prediction"]))
with Pool(processes=max(1, cpu_count() - 1), initializer=_init_scorer) as pool:
    r1_refl = np.array(
        list(
            tqdm(
                pool.imap(_r1_pair, pairs),
                total=len(eval_df_refl),
                desc="Reflexion ROUGE-1(F1)"
            )
        ),
        dtype=np.float32,
    )

# ---- MoverScore----
DEV_SKIP_MOVERSCORE = bool(int(os.getenv("DEV_SKIP_MOVERSCORE", "0")))

try:
    if not HAVE_MOVERSCORE or DEV_SKIP_MOVERSCORE:
        raise RuntimeError("MoverScore disabled or unavailable")

    MS_MAX = int(os.getenv("MS_MAX", "0"))
    eval_df_ms = (
        eval_df_refl if MS_MAX <= 0 or MS_MAX >= len(eval_df_refl)
        else eval_df_refl.iloc[:MS_MAX].copy()
    )
    refs = eval_df_ms["reference"].tolist()
    hyps = eval_df_ms["prediction"].tolist()
    idf_ref = get_idf_dict(refs)
    idf_hyp = get_idf_dict(hyps)

    ms_list = word_mover_score(
        refs, hyps, idf_ref, idf_hyp,
        stop_words=None, n_gram=1, remove_subwords=True, batch_size=64,
    )
    ms_refl = np.array([float(x) for x in ms_list], dtype=np.float32)
    if len(eval_df_ms) != len(eval_df_refl):
        ms_full = np.full(len(eval_df_refl), np.nan, dtype=np.float32)
        ms_full[:len(ms_refl)] = ms_refl
        ms_refl = ms_full
    moverscore_ok_refl = True
except Exception as e:
    print("WARNING: Reflexion MoverScore failed or skipped:", e)
    ms_refl = np.full(len(eval_df_refl), np.nan, dtype=np.float32)
    moverscore_ok_refl = False

eval_df_refl["rouge1_f1"] = r1_refl
eval_df_refl["moverscore"] = ms_refl

# ---- READABILITY ----
if textstat is not None:
    try:
        eval_df_refl["readability_fre"]   = eval_df_refl["prediction"].map(textstat.flesch_reading_ease)
        eval_df_refl["readability_fkgl"]  = eval_df_refl["prediction"].map(textstat.flesch_kincaid_grade)
        eval_df_refl["readability_smog"]  = eval_df_refl["prediction"].map(textstat.smog_index)
    except Exception as e:
        print("WARNING: Reflexion readability computation failed:", e)
        eval_df_refl["readability_fre"]  = np.nan
        eval_df_refl["readability_fkgl"] = np.nan
        eval_df_refl["readability_smog"] = np.nan
else:
    eval_df_refl["readability_fre"]  = np.nan
    eval_df_refl["readability_fkgl"] = np.nan
    eval_df_refl["readability_smog"] = np.nan

# ---- Calibration: Brier + ECE (EM-based) ----
correct_em = eval_df_refl["correctK"].astype(float).values
conf_vals  = eval_df_refl["confidence"].astype(float).values

brier_em = compute_brier(correct_em, conf_vals)
ece_em   = compute_ece(correct_em, conf_vals, n_bins=10)

summary_refl = {
    "n": int(len(eval_df_refl)),
    "rouge1_f1_mean": float(np.nanmean(r1_refl)),
    "rouge1_f1_std":  float(np.nanstd(r1_refl)),
    "moverscore_mean": float(np.nanmean(ms_refl)),
    "moverscore_std":  float(np.nanstd(ms_refl)),
    "moverscore_ok": moverscore_ok_refl,
    "ms_max": int(os.getenv("MS_MAX", "0")),
    "dev_skip_moverscore": DEV_SKIP_MOVERSCORE,
    "readability_fre_mean":   float(np.nanmean(eval_df_refl["readability_fre"])),
    "readability_fkgl_mean":  float(np.nanmean(eval_df_refl["readability_fkgl"])),
    "readability_smog_mean":  float(np.nanmean(eval_df_refl["readability_smog"])),
    "brier_em": float(brier_em),
    "ece_em":   float(ece_em),
}

out_dir_refl = CFG.OUT_DIR / f"reflexion_eval_v1_shard{CFG.SHARD_INDEX}"
out_dir_refl.mkdir(parents=True, exist_ok=True)
eval_df_refl.to_csv(out_dir_refl / "per_example_scores.csv", index=False)
with open(out_dir_refl / "summary.json", "w") as f:
    json.dump(summary_refl, f, indent=2)

print("\n[REFLEXION METRICS DONE]")
print(json.dumps(summary_refl, indent=2))


### Compile all shard results

In [11]:
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from rouge_score import rouge_scorer

# ---------- 1) Load both shards and combine ----------
path_shard0 = "/content/drive/MyDrive/11785-project-medred/outputs/medred_reflexion_v1_shard0_N250.csv"
path_shard1 = "/content/drive/MyDrive/11785-project-medred/outputs/medred_reflexion_v1_shard1_N250.csv"
path_shard2 = "/content/drive/MyDrive/11785-project-medred/outputs/medred_reflexion_v1_shard2_N251.csv"
path_shard3 = "/content/drive/MyDrive/11785-project-medred/outputs/medred_reflexion_v1_shard3_N250.csv"

df_refl0 = pd.read_csv(path_shard0)
df_refl1 = pd.read_csv(path_shard1)
df_refl2 = pd.read_csv(path_shard2)
df_refl3 = pd.read_csv(path_shard3)

df_refl0["id"] = df_refl0["id"].astype(str)
df_refl1["id"] = df_refl1["id"].astype(str)
df_refl2["id"] = df_refl2["id"].astype(str)
df_refl3["id"] = df_refl3["id"].astype(str)

df_refl_all = pd.concat([df_refl0, df_refl1, df_refl2, df_refl3], ignore_index=True)
print("Combined Reflexion rows:", len(df_refl_all))

# ---------- 2) ROUGE helper (same as before) ----------
def _init_scorer():
    global _SCORER
    _SCORER = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)

def _r1_pair(args):
    ref, hyp = args
    return _SCORER.score(ref, hyp)["rouge1"].fmeasure

def normalize(s: str) -> str:
    return " ".join((s or "").split())

# ---------- 3) Build eval_df_refl for combined shards ----------
reflex_ids = df_refl_all["id"].astype(str).unique()

df_sub = df[df["id"].astype(str).isin(reflex_ids)][["id", "reference"]].copy()
df_sub["id"] = df_sub["id"].astype(str)

refl_preds = (
    df_refl_all[["id", "predK", "correctK", "confidenceK"]]
    .rename(columns={"predK": "prediction", "confidenceK": "confidence"})
    .astype({"id": str})
)

eval_df_refl = df_sub.merge(refl_preds, on="id", how="left")
eval_df_refl["reference"]  = eval_df_refl["reference"].map(normalize)
eval_df_refl["prediction"] = eval_df_refl["prediction"].fillna("").map(normalize)

print("\nReflexion eval rows (should equal total N):", len(eval_df_refl))

# ---------- 4) ROUGE-1(F1) on combined shards ----------
pairs = list(zip(eval_df_refl["reference"], eval_df_refl["prediction"]))
with Pool(processes=max(1, cpu_count() - 1), initializer=_init_scorer) as pool:
    r1_refl = np.array(
        list(
            tqdm(
                pool.imap(_r1_pair, pairs),
                total=len(eval_df_refl),
                desc="Reflexion ROUGE-1(F1) (combined)"
            )
        ),
        dtype=np.float32,
    )

eval_df_refl["rouge1_f1"] = r1_refl

# ---------- 5) Readability on combined shards ----------
if textstat is not None:
    try:
        eval_df_refl["readability_fre"]   = eval_df_refl["prediction"].map(textstat.flesch_reading_ease)
        eval_df_refl["readability_fkgl"]  = eval_df_refl["prediction"].map(textstat.flesch_kincaid_grade)
        eval_df_refl["readability_smog"]  = eval_df_refl["prediction"].map(textstat.smog_index)
    except Exception as e:
        print("WARNING: Reflexion readability computation failed:", e)
        eval_df_refl["readability_fre"]  = np.nan
        eval_df_refl["readability_fkgl"] = np.nan
        eval_df_refl["readability_smog"] = np.nan
else:
    eval_df_refl["readability_fre"]  = np.nan
    eval_df_refl["readability_fkgl"] = np.nan
    eval_df_refl["readability_smog"] = np.nan

# ---------- 6) Calibration: Brier + ECE on combined shards ----------
correct_em = eval_df_refl["correctK"].astype(float).values
conf_vals  = eval_df_refl["confidence"].astype(float).values

brier_em = compute_brier(correct_em, conf_vals)
ece_em   = compute_ece(correct_em, conf_vals, n_bins=10)

summary_refl_combined = {
    "n": int(len(eval_df_refl)),
    "rouge1_f1_mean": float(np.nanmean(r1_refl)),
    "rouge1_f1_std":  float(np.nanstd(r1_refl)),
    "readability_fre_mean":   float(np.nanmean(eval_df_refl["readability_fre"])),
    "readability_fkgl_mean":  float(np.nanmean(eval_df_refl["readability_fkgl"])),
    "readability_smog_mean":  float(np.nanmean(eval_df_refl["readability_smog"])),
    "brier_em": float(brier_em),
    "ece_em":   float(ece_em),
}

print("\n[REFLEXION COMBINED METRICS DONE]")
print(json.dumps(summary_refl_combined, indent=2))

# save per-example + summary
out_dir_refl_all = CFG.OUT_DIR / "reflexion_eval_v1_all"
out_dir_refl_all.mkdir(parents=True, exist_ok=True)
eval_df_refl.to_csv(out_dir_refl_all / "per_example_scores.csv", index=False)
with open(out_dir_refl_all / "summary.json", "w") as f:
    json.dump(summary_refl_combined, f, indent=2)
print(f"\nSaved combined Reflexion eval to {out_dir_refl_all}")


Combined Reflexion rows: 1001

Reflexion eval rows (should equal total N): 1003


Reflexion ROUGE-1(F1) (combined): 100%|██████████| 1003/1003 [00:00<00:00, 3655.65it/s]



[REFLEXION COMBINED METRICS DONE]
{
  "n": 1003,
  "rouge1_f1_mean": 0.0959102213382721,
  "rouge1_f1_std": 0.0947948694229126,
  "readability_fre_mean": 26.562253427225347,
  "readability_fkgl_mean": 9.669453953685156,
  "readability_smog_mean": 10.133767666734183,
  "brier_em": 0.7030048661800488,
  "ece_em": 0.8326034063260342
}

Saved combined Reflexion eval to /content/drive/MyDrive/11785-project-medred/outputs/reflexion_eval_v1_all


## Reflexion V3: Short EM + last-paragraph rationale

In [ ]:
import os, gc, json, re, string, random
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Dict, Any, Optional

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm

from rouge_score import rouge_scorer
from multiprocessing import Pool, cpu_count


In [ ]:
# ------------------------------------------------------------------
# Small tokenizer compatibility patch for MoverScore (DistilBERT)
# ------------------------------------------------------------------
def _patch_tokenizer_max_len():
    try:
        import transformers
        for name in ("PreTrainedTokenizer", "PreTrainedTokenizerFast"):
            cls = getattr(transformers, name, None)
            if cls is not None and not hasattr(cls, "max_len"):
                cls.max_len = property(lambda self: getattr(self, "model_max_length", 512))
    except Exception as e:
        print("Tokenizer patch for max_len failed:", type(e).__name__, e)

_patch_tokenizer_max_len()

# ============================================================
#  Local LLM wrapper (uses existing tok + model)
# ============================================================
class LocalNotebookLLM:
    """
    Simple wrapper: llm(prompt) -> text, with max_new_tokens and stop tokens.
    Assumes `tok` and `model` are already loaded and on GPU/CPU.
    """
    def __init__(
        self,
        tokenizer,
        model,
        device: Optional[torch.device] = None,
        max_new_tokens: int = 160,
        temperature: float = 0.0,
        stop_tokens: Optional[List[str]] = None,
    ):
        self.tok = tokenizer
        self.model = model
        self.device = device or getattr(
            model,
            "device",
            torch.device("cuda" if torch.cuda.is_available() else "cpu")
        )
        self.max_new_tokens = int(max_new_tokens)
        self.temperature = float(temperature)
        self.stop_tokens = stop_tokens or [
            "\n###", "\nUser:", "\nAssistant:", "\nObservation", "Observation"
        ]

    def _apply_stop(self, text: str, stops: Optional[List[str]] = None) -> str:
        stops = stops if stops is not None else self.stop_tokens
        if not stops:
            return text
        idx = None
        for s in stops:
            if not s:
                continue
            p = text.find(s)
            if p > 0 and (idx is None or p < idx):
                idx = p
        return text[:idx] if idx is not None else text

    def __call__(self, prompt: str, max_new_tokens: Optional[int] = None) -> str:
        mnt = int(max_new_tokens) if max_new_tokens is not None else self.max_new_tokens
        max_pos = getattr(self.model.config, "max_position_embeddings", 2048)
        allowed_input = max(64, max_pos - mnt - 32)

        enc = self.tok(
            prompt,
            truncation=True,
            max_length=allowed_input,
            return_tensors="pt",
        )
        input_ids = enc["input_ids"].to(self.device)
        attn_mask = enc.get("attention_mask", None)
        if attn_mask is not None:
            attn_mask = attn_mask.to(self.device)

        with torch.inference_mode():
            out = self.model.generate(
                input_ids=input_ids,
                attention_mask=attn_mask,
                max_new_tokens=mnt,
                do_sample=(self.temperature > 0.0),
                temperature=self.temperature,
                pad_token_id=self.tok.pad_token_id,
                eos_token_id=self.tok.eos_token_id,
                use_cache=True,
            )

        gen = out[0, input_ids.shape[1]:]
        text = self.tok.decode(gen, skip_special_tokens=True)
        return self._apply_stop(text).strip()


# Instantiate LLM wrapper using your existing tok, model, CFG
llm = LocalNotebookLLM(
    tok,
    model,
    device=model.device,
    max_new_tokens=max(CFG.MAX_NEW_TOKENS, 160),
)
print("Reflexion v3 LLM ready on", model.device)

# ============================================================
#  EM helper (short answer) + confidence + calibration utils
# ============================================================
def normalize_simple(s: Optional[str]) -> str:
    if s is None:
        return ""
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def ws(text):
        return " ".join(text.split())
    txt = (s or "").lower()
    txt = remove_articles(txt)
    txt = remove_punc(txt)
    return ws(txt)

def EM(ans: str, key: str) -> bool:
    return normalize_simple(ans) == normalize_simple(key)

def extract_confidence(text: str) -> float:
    """
    Parse self-reported confidence from the model output.
    Looks for patterns like 'Confidence: 0.83' (0–1).
    Returns NaN if missing or unparsable.
    """
    if not text:
        return np.nan
    m = re.search(r"confidence\s*[:=]\s*([0-9]*\.?[0-9]+)", text, flags=re.IGNORECASE)
    if not m:
        return np.nan
    try:
        c = float(m.group(1))
        # If they give e.g. "83", treat as 0.83 (percent); clip to [0,1]
        if c > 1.0 and c <= 100.0:
            c = c / 100.0
        c = max(0.0, min(1.0, c))
        return float(c)
    except Exception:
        return np.nan

def compute_brier(correct: np.ndarray, conf: np.ndarray) -> float:
    """
    Brier score for binary outcomes.
    correct: array of 0/1
    conf   : array of p in [0,1], may contain NaN
    """
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask].astype(float)
    p = conf[mask].astype(float)
    return float(np.mean((p - y) ** 2))

def compute_ece(correct: np.ndarray, conf: np.ndarray, n_bins: int = 10) -> float:
    """
    Expected Calibration Error (ECE) using scalar self-reported confidence.
    correct: array of 0/1
    conf   : array of p in [0,1], may contain NaN
    """
    mask = ~np.isnan(conf)
    if mask.sum() == 0:
        return float("nan")
    y = correct[mask].astype(float)
    p = conf[mask].astype(float)
    N = len(p)
    bin_edges = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    for b in range(n_bins):
        left = bin_edges[b]
        right = bin_edges[b + 1]
        if b < n_bins - 1:
            in_bin = (p >= left) & (p < right)
        else:
            in_bin = (p >= left) & (p <= right + 1e-8)
        if not np.any(in_bin):
            continue
        acc = np.mean(y[in_bin])
        conf_avg = np.mean(p[in_bin])
        ece += (in_bin.sum() / N) * abs(acc - conf_avg)
    return float(ece)

class ReflexionStrategy(Enum):
    NONE = "none"
    LAST_ATTEMPT = "last_attempt"
    REFLEXION = "reflexion"
    LAST_ATTEMPT_AND_REFLEXION = "last_attempt_and_reflexion"

# ============================================================
#  CoT + Reflexion instructions (v3, with confidence line)
# ============================================================
COT_INSTRUCTION = (
    "Solve a question answering task by having a Thought, then Finish with your answer. "
    "Thought can reason about the current situation. Finish[answer] returns the answer "
    "and finishes the task. You will be given a clinical note and question.\n"
    "After you provide the Finish[answer] line, on a new separate line write "
    "'Confidence: p' where p is a number between 0 and 1 indicating how confident "
    "you are that your answer is correct.\n"
)

REFLECT_INSTRUCTION = (
    "You are an advanced reasoning agent that can improve based on self-reflection. "
    "You will be given a previous reasoning trial on a clinical QA task in which your "
    "answer was judged incorrect or low-quality. In a few sentences:\n"
    "1. Briefly explain why your reasoning or answer may have been wrong or incomplete.\n"
    "2. State 1–2 concrete rules you will follow next time to avoid the same mistake.\n"
)

COT_FEWSHOT = """Relevant Context: The Nile River is the longest river in the world, spanning approximately 6,650 kilometers (4,132 miles) in length. It flows through eleven countries in northeastern Africa, including Egypt, Sudan, and Uganda.
Question: What is the longest river in the world?
Thought: The question asks for the longest river in the world, which I know is the Nile River based on the context provided.
Action: Finish[Nile River]

Relevant Context: Ludwig van Beethoven was a German composer and pianist who was a crucial figure in the transition between the Classical and Romantic eras in Western classical music. One of his most famous compositions is the Symphony No. 9, also known as the "Choral" symphony.
Question: Which composer created Symphony No. 9?
Thought: The question is asking for the composer of Symphony No. 9. Based on the context, I know that Ludwig van Beethoven composed this symphony.
Action: Finish[Ludwig van Beethoven]

Relevant Context: Photosynthesis is the process by which green plants and some other organisms convert light energy into chemical energy. During this process, plants absorb sunlight, carbon dioxide, and water to produce glucose and oxygen.
Question: What do plants produce during photosynthesis?
Thought: The question is asking about the products of photosynthesis. From the context, I know that plants produce glucose and oxygen during this process.
Action: Finish[Glucose and Oxygen]
"""

COT_REFLECT_FEWSHOT = """
Relevant Context: Ernest Hemingway's novel "The Old Man and the Sea" tells the story of Santiago, an aging Cuban fisherman, who struggles to catch a giant marlin in the Gulf Stream. The book won the Pulitzer Prize for Fiction in 1953 and contributed to Hemingway's Nobel Prize for Literature in 1954.
Question: Which literary award did "The Old Man and the Sea" contribute to Hemingway winning?
Thought: The question is asking which award "The Old Man and the Sea" contributed to Hemingway winning. Based on the context, I know the novel won the Pulitzer Prize for Fiction and contributed to his Nobel Prize for Literature.
Action: Finish[Pulitzer Prize for Fiction]

Reflection: My answer was correct based on the context, but may not be the exact answer stored by the grading environment. Next time, I should try to provide a less verbose answer like "Pulitzer Prize" or "Nobel Prize."

Context: On 14 October 1947, Chuck Yeager, a United States Air Force test pilot, became the first person to break the sound barrier by flying the Bell X-1 experimental aircraft at an altitude of 45,000 feet.
Charles Elwood "Chuck" Yeager (13 February 1923 - 7 December 2020) was a United States Air Force officer, flying ace, and test pilot. He is best known for becoming the first person to break the sound barrier, which he achieved in the Bell X-1 aircraft named Glamorous Glennis. Yeager was also a distinguished fighter pilot during World War II and was credited with shooting down at least 12 enemy aircraft. In 1973, he was inducted into the National Aviation Hall of Fame for his significant contributions to aviation.
Question: Who is the first person to break the sound barrier?
Thought: The question is asking for the first person to break the sound barrier. From the context, I know that Chuck Yeager, a United States Air Force test pilot, was the first person to break the sound barrier.
Action: Finish[Chuck Yeager]

Reflection: Upon reflecting on the incorrect answer I provided, I realize that I may not have given the full name of the individual in question. In the context, both the given name and the nickname were mentioned, and I only used the nickname in my response. This could have been the reason my answer was deemed incorrect. Moving forward, when attempting this question again or similar questions, I will make sure to include the complete name of the person, which consists of their given name, any middle names, and their nickname (if applicable). This will help ensure that my answer is more accurate and comprehensive.
"""

# ============================================================
#  Answer & rationale extractors (Version 3, confidence-aware)
# ============================================================
def extract_medred_answer(text: str) -> str:
    """
    Short answer for EM.
    """
    if not text:
        return ""
    # 1) Action: Finish[answer]
    m = re.search(r"Action:\s*Finish\[(.*?)\]", text, flags=re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1).strip()
    # 2) Any Finish[answer]
    m = re.search(r"Finish\[(.*?)\]", text, flags=re.IGNORECASE | re.DOTALL)
    if m:
        return m.group(1).strip()
    # 3) "Final answer: ..."
    m = re.search(r"final answer\s*:\s*(.*)", text, flags=re.IGNORECASE)
    if m:
        line = m.group(1).strip().splitlines()[0]
        return line.strip()
    # 4) fallback: last non-empty line not starting with Thought/Action/Confidence
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    for line in reversed(lines):
        if re.match(r"^(thought|action|confidence)\b", line, flags=re.IGNORECASE):
            continue
        return line
    return text.strip()


def extract_rationale_last_paragraph(text: str) -> str:
    """
    Long rationale: last non-empty paragraph of the trajectory,
    ignoring pure 'Confidence: ...' paragraphs.
    """
    if not text:
        return ""
    paras = [p.strip() for p in text.split("\n\n") if p.strip()]
    if not paras:
        return text.strip()
    non_conf = [p for p in paras if not re.match(r"^confidence\b", p, flags=re.IGNORECASE)]
    if non_conf:
        return non_conf[-1]
    return paras[-1]

# ============================================================
#  MedRed CoT Reflexion agent (Version 3)
# ============================================================
@dataclass
class MedRedCoTAgent:
    ex_id: str
    query: str           # full MedRed text (title+body + question)
    answer_key: str
    llm: LocalNotebookLLM
    max_steps: int = 6
    reflexion_strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION

    # each attempt: {trajectory, answer, rationale, confidence, correct}
    attempts: List[Dict[str, Any]] = field(default_factory=list)
    reflections: List[str] = field(default_factory=list)

    def is_correct(self) -> bool:
        return bool(self.attempts and self.attempts[-1]["correct"])

    def _format_reflections_for_prompt(self) -> str:
        if not self.reflections:
            return ""
        header = (
            "You have attempted this question before and failed. "
            "The following reflection(s) give a plan to avoid failing in the same way.\n"
            "Reflections:\n"
        )
        bullets = "\n".join(f"- {r.strip()}" for r in self.reflections)
        return header + bullets

    def _build_agent_prompt(self) -> str:
        prompt = COT_INSTRUCTION
        prompt += "\nHere are some examples:\n"
        prompt += COT_FEWSHOT
        prompt += "\n(END OF EXAMPLES)\n"

        if self.reflexion_strategy in (
            ReflexionStrategy.REFLEXION,
            ReflexionStrategy.LAST_ATTEMPT_AND_REFLEXION
        ) and self.reflections:
            prompt += "\n" + self._format_reflections_for_prompt() + "\n"

        prompt += (
            "\nRelevant Context (clinical note + question):\n"
            f"{self.query.strip()}\n"
            "Question: Based on the above, what is the best answer for the patient?\n"
            "Thought 1:"
        )
        return prompt

    def step(self) -> Dict[str, Any]:
        prompt = self._build_agent_prompt()
        traj = self.llm(prompt, max_new_tokens=CFG.MAX_NEW_TOKENS + 80)

        ans       = extract_medred_answer(traj)                  # short EM answer
        rationale = extract_rationale_last_paragraph(traj)       # long rationale
        conf      = extract_confidence(traj)                     # self-reported p(correct)

        correct = EM(ans, self.answer_key)
        rec = {
            "trajectory": traj,
            "answer": ans,
            "rationale": rationale,
            "confidence": conf,
            "correct": bool(correct),
        }
        self.attempts.append(rec)
        return rec

    def reflect(self) -> Optional[str]:
        if not self.attempts:
            return None
        last = self.attempts[-1]
        refl_prompt = (
            REFLECT_INSTRUCTION
            + "\n\nHere are some reflection examples:\n"
            + COT_REFLECT_FEWSHOT
            + "\n(END OF EXAMPLES)\n"
            + "\nClinical case and question:\n"
            + self.query.strip()
            + "\n\nYour previous reasoning and answer:\n"
            + last["trajectory"]
            + "\n\nReflection:\n"
        )
        refl_text = self.llm(refl_prompt, max_new_tokens=160)
        self.reflections.append(refl_text.strip())
        return refl_text

# ============================================================
#  Fixed 1000-ID sampling + 500/500 partition
# ============================================================
MAX_STEPS = int(os.getenv("MAX_STEPS", "6"))
MAX_REFLECT_ATTEMPTS = int(os.getenv("MAX_REFLECT_ATTEMPTS", "3"))

N_EVAL_TOTAL = int(os.getenv("N_EVAL_TOTAL", "1000"))  # total MedRed eval size
SEED = int(os.getenv("SEED", "42"))
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

SHARD_TOTAL = int(os.getenv("SHARD_TOTAL", "2"))   # 2 shards of 500 by default
SHARD_INDEX = int(os.getenv("SHARD_INDEX", "1"))   # 0 or 1
assert 0 <= SHARD_INDEX < SHARD_TOTAL, "Invalid SHARD_INDEX"

# Make sure ids are strings
df_full = df.copy()
df_full["id"] = df_full["id"].astype(str)

# 1) Sample fixed 1000 IDs (or N_EVAL_TOTAL) deterministically
ids_all = (
    df_full[["id"]]
    .drop_duplicates()
    .sample(n=N_EVAL_TOTAL, random_state=SEED, replace=False)
    ["id"].astype(str).tolist()
)
print(f"[Reflexion v3] Total eval ids sampled (N_EVAL_TOTAL): {len(ids_all)}")

# 2) Partition into SHARD_TOTAL chunks
shard_size = len(ids_all) // SHARD_TOTAL
start = SHARD_INDEX * shard_size
# Last shard may take remainder if N_EVAL_TOTAL not divisible
end = (SHARD_INDEX + 1) * shard_size if SHARD_INDEX < SHARD_TOTAL - 1 else len(ids_all)

ids_shard = ids_all[start:end]
print(f"[Reflexion v3] Using shard_index={SHARD_INDEX} with rows [{start}:{end}) → {len(ids_shard)} ids")

# 3) Join with full MedRed df to get query + reference in shard order
df_reflex_in = (
    df_full.set_index("id")
           .loc[ids_shard]
           .reset_index()
)

REFLEXION_N = len(df_reflex_in)
print("Reflexion subset size for this run:", REFLEXION_N)

# ============================================================
#  Run Reflexion v3 on this shard
# ============================================================
results_reflex = []

for ex_idx, (_, row) in enumerate(
    tqdm(
        df_reflex_in[["id", "query", "reference"]].iterrows(),
        total=len(df_reflex_in),
        desc=f"Reflexion v3 episodes (shard={SHARD_INDEX})",
    ),
    start=1,
):
    agent = MedRedCoTAgent(
        ex_id=row["id"],
        query=row["query"],
        answer_key=row["reference"],
        llm=llm,
        max_steps=MAX_STEPS,
        reflexion_strategy=ReflexionStrategy.REFLEXION,
    )

    # First attempt
    first_rec   = agent.step()
    pred1       = first_rec["answer"] or ""
    rationale1  = first_rec["rationale"] or ""
    conf1       = first_rec["confidence"]
    correct1    = bool(first_rec["correct"])

    # Reflexion loop
    refl_count = 0
    while (not agent.is_correct()) and len(agent.attempts) < MAX_STEPS and refl_count < MAX_REFLECT_ATTEMPTS:
        agent.reflect()
        refl_count += 1
        agent.step()

    # Final attempt
    final_rec   = agent.attempts[-1]
    predK       = final_rec["answer"] or ""
    rationaleK  = final_rec["rationale"] or ""
    confK       = final_rec["confidence"]
    correctK    = bool(final_rec["correct"])

    results_reflex.append({
        "id": row["id"],
        "pred1": pred1,            # short answer (EM)
        "predK": predK,            # final short answer (EM)
        "rationale1": rationale1,  # long rationale for step1
        "rationaleK": rationaleK,  # long rationale for final step (metrics)
        "confidence1": conf1,      # self-reported p(correct) at step1
        "confidenceK": confK,      # self-reported p(correct) at final step
        "reference": row["reference"],
        "correct1": correct1,
        "correctK": correctK,
        "n_attempts": len(agent.attempts),
        "n_reflections": refl_count,
        "last_reflection": agent.reflections[-1] if agent.reflections else "",
    })

df_refl = pd.DataFrame(results_reflex)

# Save with shard index in the filename so you can concat later
out_name = f"medred_reflexion_v3_ids1000_shard{SHARD_INDEX}_N{REFLEXION_N}.csv"
refl_path = CFG.OUT_DIR / out_name
df_refl.to_csv(refl_path, index=False)
print("\nSaved Reflexion v3 shard outputs to", refl_path)

# Sanity check: missing short answers
missing_mask = df_refl["predK"].astype(str).str.strip() == ""
print("Missing / empty predK (short answer):", missing_mask.sum())

# EM summary (short answers) for this shard
em1 = df_refl["correct1"].mean()
emK = df_refl["correctK"].mean()
improved = ((~df_refl["correct1"]) & df_refl["correctK"]).sum()
worsened = (df_refl["correct1"] & (~df_refl["correctK"])).sum()
unchanged = len(df_refl) - improved - worsened

print("\n=== Reflexion v3 EM summary on shard (short answers) ===")
print(f"n examples           : {len(df_refl)}")
print(f"EM step1 (pred1)     : {em1:.4f}")
print(f"EM final (predK)     : {emK:.4f}")
print(f"Δ EM (final - step1) : {emK - em1:+.4f}")
print(f"Improved (False→True): {improved}")
print(f"Worsened (True→False): {worsened}")
print(f"Unchanged            : {unchanged}")

# ============================================================
#  Calibration: Brier + ECE on final short answers + confidenceK
# ============================================================
ECE_BINS = int(os.getenv("ECE_BINS", "10"))

correct_final = df_refl["correctK"].astype(float).values
conf_final    = df_refl["confidenceK"].astype(float).values

brier = compute_brier(correct_final, conf_final)
ece   = compute_ece(correct_final, conf_final, n_bins=ECE_BINS)

summary_calib = {
    "n": int(len(df_refl)),
    "em1_mean": float(em1),
    "emK_mean": float(emK),
    "brier": float(brier),
    "ece": float(ece),
    "ece_bins": ECE_BINS,
}

summary_path = CFG.OUT_DIR / f"reflexion_v3_ids1000_shard{SHARD_INDEX}_calibration_summary.json"
with open(summary_path, "w") as f:
    json.dump(summary_calib, f, indent=2)

print("\n[REFLEXION v3 CALIBRATION SUMMARY]")
print(json.dumps(summary_calib, indent=2))


### Combine shard 0,1

In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count
from rouge_score import rouge_scorer

# ---------- 1) Load both shards and combine ----------
path_shard0 = "/kaggle/input/reflexion-long/medred_reflexion_v3_shard0_N500.csv"
path_shard1 = "/kaggle/input/reflexion-long/medred_reflexion_v3_shard1_N501.csv"


df_refl0 = pd.read_csv(path_shard0)
df_refl1 = pd.read_csv(path_shard1)


# Make sure ids are strings
df_refl0["id"] = df_refl0["id"].astype(str)
df_refl1["id"] = df_refl1["id"].astype(str)


df_refl_all = pd.concat([df_refl0, df_refl1], ignore_index=True)
print("Combined Reflexion rows:", len(df_refl_all))

# ---------- 2) ROUGE helper (same as before) ----------
def _init_scorer():
    global _SCORER
    _SCORER = rouge_scorer.RougeScorer(["rouge1"], use_stemmer=True)

def _r1_pair(args):
    ref, hyp = args
    return _SCORER.score(ref, hyp)["rouge1"].fmeasure

def normalize(s: str) -> str:
    return " ".join((s or "").split())

# ---------- 3) Build eval_df_refl for combined shards ----------
reflex_ids = df_refl_all["id"].astype(str).unique()

# df is your original MedRed dataframe with id + reference
df_sub = df[df["id"].astype(str).isin(reflex_ids)][["id", "reference"]].copy()
df_sub["id"] = df_sub["id"].astype(str)

# Bring in prediction, correctness, and confidence from predK
refl_preds = (
    df_refl_all[["id", "predK", "correctK", "confidenceK"]]
    .rename(columns={"predK": "prediction", "confidenceK": "confidence"})
    .astype({"id": str})
)

eval_df_refl = df_sub.merge(refl_preds, on="id", how="left")
eval_df_refl["reference"]  = eval_df_refl["reference"].map(normalize)
eval_df_refl["prediction"] = eval_df_refl["prediction"].fillna("").map(normalize)

print("\nReflexion eval rows (should equal total N):", len(eval_df_refl))

# ---------- 4) ROUGE-1(F1) on combined shards ----------
pairs = list(zip(eval_df_refl["reference"], eval_df_refl["prediction"]))
with Pool(processes=max(1, cpu_count() - 1), initializer=_init_scorer) as pool:
    r1_refl = np.array(
        list(
            tqdm(
                pool.imap(_r1_pair, pairs),
                total=len(eval_df_refl),
                desc="Reflexion ROUGE-1(F1) (combined)"
            )
        ),
        dtype=np.float32,
    )

eval_df_refl["rouge1_f1"] = r1_refl

# ---------- 5) Readability on combined shards ----------
# Make sure textstat is imported and not None:
#   import textstat
#   (or your earlier try/except that sets it)
if textstat is not None:
    try:
        eval_df_refl["readability_fre"]   = eval_df_refl["prediction"].map(textstat.flesch_reading_ease)
        eval_df_refl["readability_fkgl"]  = eval_df_refl["prediction"].map(textstat.flesch_kincaid_grade)
        eval_df_refl["readability_smog"]  = eval_df_refl["prediction"].map(textstat.smog_index)
    except Exception as e:
        print("WARNING: Reflexion readability computation failed:", e)
        eval_df_refl["readability_fre"]  = np.nan
        eval_df_refl["readability_fkgl"] = np.nan
        eval_df_refl["readability_smog"] = np.nan
else:
    eval_df_refl["readability_fre"]  = np.nan
    eval_df_refl["readability_fkgl"] = np.nan
    eval_df_refl["readability_smog"] = np.nan

# ---------- 6) Calibration: Brier + ECE on combined shards ----------
# reuse your existing compute_brier / compute_ece
correct_em = eval_df_refl["correctK"].astype(float).values
conf_vals  = eval_df_refl["confidence"].astype(float).values

brier_em = compute_brier(correct_em, conf_vals)
ece_em   = compute_ece(correct_em, conf_vals, n_bins=10)

summary_refl_combined = {
    "n": int(len(eval_df_refl)),
    "rouge1_f1_mean": float(np.nanmean(r1_refl)),
    "rouge1_f1_std":  float(np.nanstd(r1_refl)),
    "readability_fre_mean":   float(np.nanmean(eval_df_refl["readability_fre"])),
    "readability_fkgl_mean":  float(np.nanmean(eval_df_refl["readability_fkgl"])),
    "readability_smog_mean":  float(np.nanmean(eval_df_refl["readability_smog"])),
    "brier_em": float(brier_em),
    "ece_em":   float(ece_em),
}

print("\n[REFLEXION COMBINED METRICS DONE]")
print(json.dumps(summary_refl_combined, indent=2))

# Optional: save per-example + summary
out_dir_refl_all = CFG.OUT_DIR / "reflexion_eval_v3_all"
out_dir_refl_all.mkdir(parents=True, exist_ok=True)
eval_df_refl.to_csv(out_dir_refl_all / "per_example_scores.csv", index=False)
with open(out_dir_refl_all / "summary.json", "w") as f:
    json.dump(summary_refl_combined, f, indent=2)
print(f"\nSaved combined Reflexion eval to {out_dir_refl_all}")
